In [3]:
import csv
import pandas as pd
import re
import numpy as np
import pickle
import torch
from transformers import AutoTokenizer

/home/lomboa00/cnnt/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Some functions for data processing that will be used later

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('camembert-bio-model')

In [ ]:
def categorical(prob, n_samples):
    """
    sample a categorical distribution from a vect of probabilities
    """
    prob = prob.unsqueeze(0).repeat(n_samples, 1)
    cum_prob = torch.cumsum(prob, dim=-1)
    r = torch.rand(n_samples, 1)
    # argmax finds the index of the first True value in the last axis.
    samples = torch.argmax((cum_prob > r).int(), dim=-1)
    return samples.numpy()

In [ ]:
def clean_data(data):
    if (len(data) > 0) and (data[-1] == ' '):
        return data[:-1]
    return data

In [ ]:
def remove_punctuation(word):
    #print(word, len(word))
    if len(word) == 0:
        return []
    if len(word) == 1:
        return [word]
    
    if word[0] in [':', ',', '.']:
        return [word[0]] + remove_punctuation(word[1:])
    
    idx = 0
    while (idx < len(word)) and (not word[idx] in ['!', '(', ')', ';']) :
        idx += 1
    
    if idx == 0:
        return [word[0]] + remove_punctuation(word[1:])
    if idx == len(word):
        if word[-1] in ['.', ',', ':']:
            return [word[:-1], word[-1]]
        else:
            return [word]
    
    return [word[:idx], word[idx]] + remove_punctuation(word[idx+1:])

In [ ]:
#auxiliary function for the custom tokenizer
def aux(word):
    cap_symbols = ['T', 'P', 'F', 'B', 'A', 'C', 'D', 'U'] #['T', 'P', 'G', 'L', 'M', 'C', 'D', 'U'] for v1
    small_symbols = ['d', 'c', 'a', 'f', 'b', 'u', 'p', 'n'] #['d', 'c', 'm', 'f', 'l', 'u', 'p', 'n'] for v1
    
    result = ''
    # find if there are decimals
    has_decimals = False
    if ('.' in word): #priority to '.', cause no ambiguity
        comma_idx = word.index('.')
        has_decimals = True
        if (',' in word):
            print('error ! dot and comma in number')
            
    elif (',' in word):
        comma_idx = word.index(',')
        has_decimals = True
    
    n=len(word)-1
    
    idx = n
    if has_decimals: #tokenizes the decimal part
        while idx > comma_idx:
            result = word[idx] + small_symbols[(idx-comma_idx-1)] + result
            idx -= 1
        n = comma_idx - 1
    
    if n > 7:
        return ""
    idx = 0
    while idx <= n: #tokenizes the main part
        result = word[n-idx] + cap_symbols[-(idx+1)] + result
        idx += 1
    return result

# 2. Various number formatting and cleaning

## 2.1. Here we define our custom tokenizer designed to handle numbers by breaking them down.
See the bank of tests below for a more detailed understanding

In [ ]:
def custom_tokenizer(word):  
    #isolate numbers
    #number surounded  
    result = re.sub(r"([^\d])(\d+[\.,]\d+)([^\d])", r"\1 \2 \3", word)
    result = re.sub(r"([^\d\.,\s])(\d+)([^\d\.,\s])", r"\1 \2 \3", result)
    
    result = re.sub(r"([^\d])(\d+[\.,]\d+)([^\d])", r"\1 \2 \3", result)
    result = re.sub(r"([^\d\.,\s])(\d+)([^\d\.,\s])", r"\1 \2 \3", result)
    
    #number at the beginning
    result = re.sub(r"(^|\s)(\d+[\.,]\d+)([^\d\s])", r"\1\2 \3", result)
    result = re.sub(r"(^|\s)(\d+)([^\d\.,\s])", r"\1\2 \3", result)

    #number at the end
    result = re.sub(r"([^\d\s])(\d+[\.,]\d+)(\s|$)", r"\1 \2\3", result)
    result = re.sub(r"([^\d\.,\s])(\d+)(\s|$)", r"\1 \2\3", result)
    
    #tokenize isolated numbers
    result = re.sub(r"(^|\s)(\d+[\.,]\d+)(\s|$)", lambda x: x.group(1)+aux(x.group(2))+x.group(3), result)
    result = re.sub(r"(^|\s)(\d+)(\s|$)", lambda x: x.group(1)+aux(x.group(2))+x.group(3), result)
    
    #cleaning (ex -/ or /-)
    result = re.sub(r"\s", r"-", result)
    result = re.sub(r"-([^A-Za-z0-9><])", r"\1", result)
    result = re.sub(r"([^A-Za-z0-9><])-", r"\1", result)
    
    return result

### 2.1.1. Bank of tests for the custom tokenizer

In [8]:
custom_tokenizer("G2P2")

'G-2U-P-2U'

In [9]:
custom_tokenizer("G4P2A1")

'G-4U-P-2U-A-1U'

In [10]:
custom_tokenizer("22q11.5")

'2D2U-q-1D1U5d'

In [11]:
custom_tokenizer("cm2/m3")

'cm-2U/m-3U'

In [12]:
custom_tokenizer("Spo2")

'Spo-2U'

In [13]:
custom_tokenizer("123.56")

'1C2D3U5d6c'

## 2.2. Here we implement a function to handle artefacts and noise in the text.
See the bank of tests examples for a better understanding of what this function does

In [14]:
def intermediate(word):
    # deals with apostrophes at the end of numbers
    result = re.sub(r"(\d)'$", r"\1", word)
    #convert scientific notation to arabic notation
    result = re.sub(r"(^|\s)(\d+e\d+|\d+E\d+|\d+e-\d+|\d+E-\d+)", lambda x: x.group(1)+str(float(x.group(2))), result)
    #creates a space separation for percentages
    result = re.sub(r"(\d)%", r"\1 %", result)
    #separates equalites (FR=50)
    result = re.sub(r"=([^\s])", r" = \1", result)
    # deals with certain codes 2q11.5q12-->2q11.5 - 2q12
    result = re.sub(r"(^|\s)(\d+)(q|p)(\d+[\.,]\d+|\d+)-?(p|q|/)(\d+[\.,]\d+|\d+)", r"\1\2\3\4 - \2\3\6", result)
    result = re.sub(r"(^|\s)(\d+)(q|p)(\d+[\.,]\d+|\d+)-?(p|q|/)(\d+[\.,]\d+|\d+)", r"\1\2\3\4 - \2\3\6", result)
    result = re.sub(r"(^|\s)(\d+)(q|p)(\d+[\.,]\d+|\d+)-?(p|q|/)(\d+[\.,]\d+|\d+)", r"\1\2\3\4 - \2\3\6", result)
    result = re.sub(r"(^|\s)(\d+)\.(q|p)\.(\d+[\.,]\d+|\d+)", r"\1\2\3\4", result)
    
    #separates range of numbers (except apgar) and operations (32+6)
    result = re.sub(r"(\d|%)(-+|\++|/+|:+|x+|X+|\|+|>+|<+|\*|~)(\d)", r"\1 \2 \3", result)
    #runs again in case there is apgar or a date
    result = re.sub(r"(\d|%)(-+|\++|/+|:+|x+|X+|\|+|>+|<+|\*|~)(\d)", r"\1 \2 \3", result)

    # deals with certain numbers representation 2kg48-->2.48kg
    result = re.sub(r"(^|\s)(\d+)(kg|KG|Kg)(\d+)", r"\1\2.\4\3", result)
    # deals with time representation 2h48-->2 h 48
    result = re.sub(r"(^|\s)(\d+)(h|H)(\d+)", r"\1\2 \3 \4", result)
    
    #specifically targets apgar (8.8.8) and dates
    result = re.sub(r"^(\d+)(\.|,)(\d+)(\.|,)(\d+)", r"\1 - \3 - \5", result)
    #deals with lists
    result = re.sub(r"(\s|^)(\d+[\.)])([A-Za-zÀ-ÿ]{2,})(\s|$)", r"\1item) \3\4", result)
    
    # remove noise at the end of numbers (12-->)
    result = re.sub(r"(^|\s)(\d+[\.,]\d+)\.([^A-Za-z0-9\s]+)", r"\1\2 \3", result)
    result = re.sub(r"(^|\s)(\d+[\.,]\d+)([^A-Za-z0-9\s\.]+)", r"\1\2 \3", result)
    if not re.search(r"(^|\s)(\d+[\.,]\d+)", result): #when the format d,d is not detected
        result = re.sub(r"(^|\s)(\d+)\.([^A-Za-z0-9\s]+)", r"\1\2 \3", result)
        result = re.sub(r"(^|\s)(\d+)([^A-Za-z0-9\s\.]+)", r"\1\2 \3", result)
        
    result = re.sub(r"(\d)(:|/)([A-Za-z])", r"\1 \2 \3", result)
    result = re.sub(r"(\d),([A-Za-z])", r"\1 , \2", result)
    result = re.sub(r"(\d)\.([A-Za-pr-z])", r"\1 \2", result)
    # remove noise at the beginning of numbers (-->12)
    result = re.sub(r"(\s|^)([^A-Za-z0-9\s]+)(\d+[\.,]\d+|\d+)", r"\1\2 \3", result)
    result = re.sub(r"([A-Za-z]+),(\d)", r"\1, \2", result)
    result = re.sub(r"([A-Za-pr-z]+)\.(\d)", r"\1 \2", result)
    result = re.sub(r"([A-Za-pr-z]+)(\d+[\.,]\d+)", r"\1 \2", result)
    result = re.sub(r"(^|\s)(TA|TVC|FC|Fc|fc|min|q|Q)(\d)", r"\1\2 \3", result) 
    
    #separates numbers and units
    result = re.sub(r"(\s|^)(\d+[\.,]\d+|\d+)([A-Za-pr-z])", r"\1\2 \3", result)
    # deals with operations within texts (x12 or SA+2jr)
    result = re.sub(r"(x|\+|X|:|<|>|/|-|«)(\d)", r" \1 \2", result)
    result = re.sub(r"([A-Za-z0-9])->", r"\1 ->", result)
    result = re.sub(r"->([A-Za-z0-9])", r"-> \1", result)
    
    # deals with LetterDigit codes (J2, ccq3h)
    result = re.sub(r"(^|\s)([JGSjgs])(\d+)(\s|$)", r" \1\2 \3\4", result)
    result = re.sub(r"ccq(\d+)", r" cc q \1", result)
    # remove noise at the beginning of words (S02->SO2)
    result = re.sub(r"(^|\s)(-|\.|,|\+)([A-Za-z])", r"\1\2 \3", result)
    # remove noise at the beginning of words (-SO2)
    result = re.sub(r"([^0-9])02(\s|$)", r"\1O2\2", result)
    # remove noise at the end of words (SO2-)
    result = re.sub(r"([^\s])-(\s|$)", r"\1 - \2", result)

    return result

#we run it twice to ensure all the issues are solved
def minor_edit(word):
    edited_word = intermediate(word).split()
    result = []
    for token in edited_word:
        result.append(intermediate(token))
    return ' '.join(result)

### 2.2.1. Bank of tests for the artefacts remover

In [15]:
minor_edit("100-115%")

'100 - 115 %'

In [16]:
minor_edit("9-8-9")

'9 - 8 - 9'

In [17]:
minor_edit("20%>30%")

'20 % > 30 %'

In [18]:
minor_edit("8h30")

'8 h 30'

In [19]:
minor_edit("22h50min")

'22 h 50 min'

In [20]:
minor_edit("SpO2")

'SpO2'

In [21]:
minor_edit("23,2cm")

'23,2 cm'

In [22]:
minor_edit("2m")

'2 m'

In [23]:
minor_edit("14/08")

'14 / 08'

In [24]:
minor_edit("1.23:23,1||12")

'1.23 : 23,1 || 12'

In [25]:
minor_edit("2m - 3m")

'2 m - 3 m'

In [26]:
minor_edit("179'")

'179'

In [27]:
minor_edit("32+6")

'32 + 6'

In [28]:
minor_edit("14.5,svo2")

'14.5 , svo2'

In [29]:
minor_edit('2q11')

'2q11'

In [30]:
minor_edit("2.q.11")

'2q11'

In [31]:
minor_edit("22.q.11/Syndrome")

'22q11 / Syndrome'

In [32]:
minor_edit("19q13/42q13/43")

'19q13 - 19q42 - 19q13 - 19q43'

In [33]:
minor_edit('1q21.1q21.2')

'1q21.1 - 1q21.2'

In [34]:
minor_edit('2q32.1-q34')

'2q32.1 - 2q34'

In [35]:
minor_edit("50ccq3h")

'50 cc q 3 h'

In [36]:
minor_edit("q3-4h q3h et q2sem")

'q 3 - 4 h q 3 h et q 2 sem'

In [37]:
minor_edit("q20min->q1h->q2h")

'q 20 min -> q 1 h -> q 2 h'

In [38]:
minor_edit("12-->14")

'12 --> 14'

In [39]:
minor_edit("7,19/6-->")

'7,19 / 6 -->'

In [40]:
minor_edit("-9-9")

'- 9 - 9'

In [41]:
minor_edit("12.67/2,3:7/min/1.23mm2")

'12.67 / 2,3 : 7 / min / 1.23 mm2'

In [42]:
minor_edit("2.8x3,5x87cm")

'2.8 x 3,5 x 87 cm'

In [43]:
minor_edit("2kg48")

'2.48 kg'

In [44]:
minor_edit("2.2.9")

'2 - 2 - 9'

In [45]:
minor_edit("2/kg")

'2 /kg'

In [46]:
minor_edit("2:G")

'2 :G'

In [47]:
minor_edit("<2s")

'< 2 s'

In [48]:
minor_edit("1.encephalogramme")

'item) encephalogramme'

In [49]:
minor_edit("369mm2/m2")

'369 mm2 / m2'

In [50]:
minor_edit('11h00-05h00-->')

'11 h 00 - 05 h 00 -->'

In [51]:
minor_edit("5e6")

'5000000.0'

In [52]:
minor_edit("-FR=50 =FC130")

'- FR = 50 = FC 130'

In [53]:
minor_edit("x20/sem")

'x 20 / sem'

In [54]:
minor_edit("J1")

'J 1'

In [55]:
minor_edit("N<35mm/m2")

'N < 35 mm/m2'

## 2.3. Here we define two functions designed to mask numbers.

`number_masking` is used for preprocessing in the
[first paper](https://arxiv.org/abs/2404.10171)

`number_blinding`is used for preprocessing in the [second paper](https://arxiv.org/html/2405.18448v2)

See the bank of test examples for a better understanding of how these function do

In [56]:
#returns the blinded word with a number corresponding to its coefficient (see Xval paper)
def number_blinding(word):
    #single number
    if re.search(r"^(\d+[,\.]\d+|\d+\.?)$", word):
        #1 is very often used as a determinant than a number
        if word == "1":
            return word, 1
        return "NUM", float(re.sub(r",", r".", word))
    if re.search(r"^(\d+[,\.]\d+|\d+)\.$", word):
        return "NUM", float(re.sub(r",", r".", word[:-1]))

    #if re.search(r"\d", word) and not re.search(r"O2$", word):
    #    print(word, custom_tokenizer(word))
    return word, 1
    

In [57]:
#returns the blinded word
def number_masking(word):
    #single number
    if re.search(r"^(\d+[,\.]\d+|\d+\.?)$", word):
        #1 is very often used as a determinant than a number
        if word == "1":
            return word
        return "nombre"
    if re.search(r"^(\d+[,\.]\d+|\d+)\.$", word):
        return "nombre"

    #if re.search(r"\d", word) and not re.search(r"O2$", word):
    #    print(word, custom_tokenizer(word))
    return word
    

### 2.3.1. Bank of test examples for number_masking and number_blinding

In [6]:
number_blinding("SpO2")

('SpO2', 1)

In [58]:
number_masking("SpO2")

'SpO2'

In [7]:
number_blinding("1")

('1', 1)

In [59]:
number_masking("1")

'1'

In [8]:
number_blinding("8")

('NUM', 8.0)

In [60]:
number_masking("8")

'nombre'

In [9]:
number_blinding("12,15")

('NUM', 12.15)

In [61]:
number_masking("12,15")

'nombre'

In [10]:
number_blinding("12,0.")

('NUM', 12.0)

In [62]:
number_masking("12,0.")

'nombre'

## 2.4. This function performs scientific notation formatting

In [63]:
#returns significand and exponent
def scientific_notiation(num):
    result = "{:e}".format(num)
    exponent = re.sub(r"^[\d\.]*e([\+-]\d+)$", r"\1", result)
    significand = re.sub(r"^([\d\.]*)e[\+-]\d+$", r"\1", result)
    return float(significand), int(exponent)

In [64]:
scientific_notiation(0.12)

(1.2, -1)

In [65]:
scientific_notiation(2013)

(2.013, 3)

# 3. Preprocessing the documents

## 3.1. Opening documents
Use the right encoding and some useless characters

Before running these lines, create a sadcsip folder and put the two data files `NLP_data/2020.06.03_CHUSJ_Data_PatientID.csv`and `NLP_data/Labelling Le - 0 to 100.csv`in it

In [73]:
with open("sadcsip/NLP_data/2020.06.03_CHUSJ_Data_PatientID.csv", 'r', encoding = 'ISO-8859-1') as file: #latin-1 delimiter = '\t'
    csvreader = csv.reader((line.replace('\0','').replace('\t-', '').replace('\t', '').replace(' \x19', "'") for line in file))
    notes = []
    for row in csvreader:
        notes.append(row)

In [74]:
with open("sadcsip/NLP_data/Labelling Le - 0 to 100.csv", 'r', encoding = 'utf-8') as file:
    csvreader = csv.reader((line.replace('\0','') for line in file))
    labels = []
    for row in csvreader:
        row[2] = clean_data(row[2])
        labels.append(row)

## 3.2. Cleaning some minor text mistakes 

In [75]:
notes[20][2] = 'Né à 41 semaines, 1ère grossesse, pas complication C/s élective, Apgar 8-9 Cardiopathie dépistée en anténatal. Amnio: CGH normal Écho le 23.01: FOP 3-4 mm. Hypoplasie VG. Atrésie mitrale. Pas IT. CIV musc. inlet 5,5 mm.VDDI. Atrésie pulmonaire avec plancher valve pulmonaire non perforé bombant sous v. Ao. Valve Ao remaniée sans fuite significative, gradient 9 mmHg. CA large 3,5 mm shuntant G-D non restrictif. Branches pulmonaires tailles normales. Hypoplasie anneau et APP 4,5mm. Pas de flux visualisé VD à APP. distribution coronarienne N. Arc Ao G normal. Rashkind le 25.01 Sous prota 0,025mcg/kg/min depuis la naissance KT le 27.01: échec dilatation v.pulmo. épanchement péricardiaque post-ponction  Vu par génétique: cardiopathie isolée, CGH -, pas autre test nécessaire'

In [76]:
notes[46][2] = " DAN de TOF à 23 SA.  Naissance à 40 SA. PN 2kg820 PC 30-32 cm    Tétralogie fallot extrême       -29/07/2011, à J10patch transannulaire sans fermeture de CIV.       -12/09/2011, à 2 mois de vie, correction chirurgicale complète : patch reprise et fermeture de la CIV. Chylothorax en post opératoire.       -07/11/2011, à 4 mois de vie, angioplastie par stents des 2 artères pulmonaires.       -03//02/2012, à 7 mois, nouveau KT pour défaillance cardiaque droite et PVD supra-systémique. On réalise des angioplasties multiples des artères pulmonaires proximales (a/n stent APG et stent APD) et lobaires avec légère amélioration des pressions dans le VD mais qui demeurent isosystémiques. PVD 74 mmHg. G° VD-APP distal 28mmHg.       -Mise en évidence de resténose des APs avec PVD suprasystémique motivant une nouveau KT le 11/06/2012 à 11 mois.          Patient reste avec pressions VD isosystémique; toutefois la plupart du gradient semble provenir des stents qui sont à leur capacité maximale de dilatation                    percutanée secondaire à prolifération intimale.  Présence d'un anévrysme de l'APG (LIG) de novo après dernière dilatation et qui devra être suivi par imagerie.          (PressionS VD 95 AP 83 Pré stent 79 Post stent 45) Dernière écho cardique 09/04/2013 TOF extreme. S/P correction chirurgicale. SPP bilatérale S/P implantation de stents. Post dilatation par ballon des stents des 2 AP. SPP bilatérale S/P implantation de stents. Patch de CIV sans évidence de résiduelle. Pas d`IAo. Légère accélération dans la voie de chasse du VD ( 15mmHg) . IP légère à modérée avec un reflux diastolique uniquement dans l`APD ( pas de reflux dans l`APP et l`APG). Stents a/n des branches pulmonaires visualisés. Branche droite proximale mesurée à 4.2 mm. Branches gauche mesurée 4.2 mm. Gradient non mesuré a/n des branches  IT légère excentrique, gradient VD-OD de 85 à 90 mmHg  Courbure septale de type II + en systole et en diastole. Bonne contractilité VG. .  VD dilatée et hypertrophié. Fonction VD diminuée qualitativement  Stridor Post 3 extubation (07/2011, 09/2011, 11/2011)    Scopie le 11/2011: Oedème de la glotte    Bronchoscopie rigide 03/02/2012 : N    Pas de lésion de la ss-glotte  Hospit X2 déc 2012 Bronchospasme/Laryngo-trachéo-bronchite  Microcéphalie/Retard de croissance (déjà vu en génétique à la naissance)   FISH 22q11 nég "

In [77]:
notes[174][2] = 'naissance 37 +5 PN 3.2 D TGV connue Rashkind J1, CIA 7.2 mm Poursuite Pg 0.02 Defaillance x 22.02.2013: CPAP, LANOXIN, LASIX. Tchypnée, pas de tirage, pas de lactates'

In [78]:
notes[300][2] = "1) Née à 38 2/7 semaines. PN 2900 g Diagnostic anténatal de cardiopathie complexe avec T4F et CAVC. APGAR 9 - 9 - 9. SpO2 la naissance 75-80% Mise rapidement sous Inderal.   2) Écho post natale: (S,D,S). Situs solitus. Lévocardie. Arc aortique gauche. Concordance AV et VA. Retour veineux pulmonaire normal. Canal AV complet type A de Rastelli shuntant G-D. Valve AV commune balancée avec insuffisance légère. TOF avec sténose pulmonaire mixte infundibulaire et valvulaire avec CIV de 5.25 mm par déviation antérieure du septum conal. Gradient VD-APP pic de 30 mmHg (sténose infundibulaire en lame de sabre à 9 mmHg). VCSG se drainant dans l`oreillette gauche. Petite VCSD drainant dans l`oreillette droite, sans TVI. CIA très large confinant à l`oreillette unique. petit SIA résiduel. Petit canal artériel perméable tortueux à shunt G-D avec gradient max à 17 mmHg. VCI visualisée, sans interruption.   3) Dilatation de la valve pulmonaire le 03/05/2013 Contexte de désaturation avec besoin en oxygène et majoration du gradient VD AP de 30 à 73mmHg (CA petit non contributif). Évolution post KT amélioration progressive des besoins en O2 jusqu'à sevrage Congés avec Sat entre 75-85% en AA  4) Sept 2013 et janvier 2014 : Suspicion de bactériémie à Staphylocoques epidermidis et S saprophyticus en contexte fébrile. Traitement par Cefotaxime puis cloxacilline. hemocultures de contrôle negatives avec  bilan infectieux normal évoquant une contamination, amélioration avec hydratation et évolution.  5) Mars 2014: IVRS avec hemoc négative, sans surinfection pulmonaire d'évolution favorable après 48h de cefotaxime.  *** Bilan pré-op: - ECG (11/02/2014) Rythme atrial ectopique, hypertrophie bi ventriculaire (T positive en V1 et S profonde en V1 V2) QRS fins. Axe QRS à  \x13 50. PR normal, QT  normal - Rx: Cardiomégalie avec débord de l'arc inférieur droit, vascularisation pulmonaire normale, silhouette thymique, pas d'anomalies pleuroparenchymateuses - Echographie cardiaque (11/02/2014): Tétralogie de Fallot avec CAVC  s/p dilatation de la valve pulmonaire par KT interventionnel. CIV large de 9.4 mm shuntant bi-directionnel, mais prédominance D-G. Sténose valvulaire pulmonaire, infundibulaire avec hypoplasie de l`APP, gradient VD-AP total de 51 mmHg.  Branches pulmonaires proximales visualisées en vue sous-costale et semblent de bon calibre. Insuffisance pulmonaire discrète. Canal artériel non visualisé. Fonction cardiaque qualitativement normale. VCSG avec hémi-azygos drainant dans OG non recherchée ce jour. Oreillette unique. Fuite légère sur VAV commune. - Scanner cardiaque (18/02/2014) Fallot avec canal AV complet et situs ambigu (asplénie et deux poumons de configuration gauche, deux veines caves supérieures). Hypertrophie ventriculaire droite en lien avec une probable obstruction pulmonaire mal visible. CIV de l'inlet et de l'outlet. Oreillette unique. Absence d'anneau vasculaire. Hypoplasie pulmonaire avec une artère pulmonaire principale mesurée à 7,4 mm, une branche droite à 8,5 mm et une branche gauche à 8,1 mm.   Soit NAKATA calculé 283.35"

In [79]:
notes[122][2] = "Naissance terme, APGAR 7-9-10 PN 2470 Tétralogie de Fallot découverte à la visite de routine de 3 semaines de vie (prise de poids difficile, SaO2 85-90% aa). Mise initialement sous prostaglandines et transférée à Sainte-Justine. Dx tétralogie de Fallot extrême avec arc aortique D.  KT cardiaque 2013-04-03 : Débit cardiaque 4,41L/min/m2. Tétralogie de Fallot extrême . Arc aortique D. Artère sous claviève G abérente. Pas de CA. Implantation et trajet des coronaires N. Atrésie pulmonaire confirmée avec nombreuses collatérales aorto pulmonaire.  CT scan 2013-03-29 : Arc Ao droit avec sous-clavière gauche aberrante. Pas de diverticule de Kommerell. ToF avec AP de 2.9 mm. AP primitives filiformes mais continuité, nettement hypoplasiques. AP droite 1,6 mm; AP gauche 2.1 mm. Multiples collatérales Aorto-pulm (5). Retour veineux pulmonaire normal au niveau OG.  Ao augmenté de volume avec Ao chevauchante et hypertrophie venticulaire droite significative. 2 petites CIV (mid-musculaire et apex). Signes hyperinflation LM prob secondaire à multiples collatérales qui encerclent la bronche lobaire moyenne. Zones d'atélectasie importante en LSD, LIG, LID et paramédian D au niveau segement supéro-dosrsal du LID. Dernière écho cardiaque 2013-05-15 : Atrésie pulmonaire uniquement valvulaire avec CIV. Valve pulmonaire de 3.48mm sans flux antérograde VD-AP. APD= 2mm. APG = 2.6mm.  CIV membraneuse a extension conale de 10mm avec shunt non restrictif bidrectionnel mais presque exclusivement D-G. Petite CIV mid-musculaire addiotionnelle. Pas de CIV apicale visualisée. IAo minime. Pas d`accélération VG-Ao. Bonne fonction bi-ventriculaire. TAPSE=11.6mm. S`VD=12cm/s. Pas d`épanchement péricardique. Arc aortique droit, sous claviere gauche aberrante. Présence de collatérales aorto-pulmonaires a/n de l`aorte descendante thoracique. Flux rétrograde diastolique a/n de l`aorte abdominale. FOP non visualisée."

In [80]:
notes[96][2] = "Atresie tricuspide, hypoplasie VD, L-Malposition des Gros Vaisseaux, hypoplasie aorte transverse sévère S/P Norwood stade I 8j de vie S/P Dilatation pour re-coarctation 2 mois de vie S/P 11/2010:            -Stent pré-monté Genesis 7x12mm (échec de montée de stent Genesis 1910 a/n voie fémorale 6fr)           -Embolisations de 3 collatérales A-P par de multiples coils; embolisation d'un coil a/n de l'artère brachiale gauche.  S/P Glenn 11/2010 S/P 05/2012:           -Dilatation du stent aortique par ballon Powerflex de 12x2 avec dilatation du stent jusqu'à 9.1mm           -ATL de l'artère lobaire supérieure droite           -Par ailleurs: collatérales V-V a/n du TVI à emboliser avant Fontan  Au dossier de la réunion médico-chx de cardiologie du 2013-09-09: Candidat à une dérivation bicavo-pulmonaire. Indice de Nakata de 148mm2/m2 avec scénario du mieux (au KT du 08/2013) Embolisation de l'AMID n'est pas indiquée vu les PAPm Indication de fenestration retenue.  Dernière Écho cardiaque (2014-06-26): Atrésie tricuspide et TGV (S,D,L) s/p DKS, BDG et stents dans Ao transverse. S/P fermeture collaterales veno-veineuses et re dilatation stent crosse aortique. Large CIA chirurgicale  shuntant D-G majoritairement.  Fonction diastolique:  IPM du VG de 0.42 par Doppler tissulaire.  Onde S` du VG à 6.82 cm/s. Bonne contractilité ventriculaire gauche qualitative. Simpson de 65%. Pas d`insuffisance de la valve mitrale. Insuffisance légère à modérée de la néoaorte avec renversement du flot  en proto-diastole dans l`aorte abdominale. Glenn bien perméable et flux laminaire et phasique. Aorte native et DKS bien visualisé en sous-xyphoidienavec flux laminaire. Arc aortique avec stent visualisé, gradient de 13 mmHg sans extension diastolique. Pas d`extension diastolique a/n de l`aorte abdominale. Pas d`épanchement péricardique.  Patient stable à domicile. Sous ASA 80 à la maison."

In [81]:
notes[252][2] = "11 mois  diagnostic anténatal de de D-TGV. Apgar 8-9-9 Né à 38 4/7 sem, Diagnostic confirmé d-tgv avec large CIV. Déviation postérieur du septum conal en sous pulmonaire avec accélération sous valvulaire et valvulaire pulmonaire.  Pas de atrioseptostomie nécessaire. IRMc : VDDI avec malposition des gros-vaisseaux et sténose pulmonaire. Opéré à 10 jours de vie : Ligature et division du CA, Résection sous néo-aortique, Fermeture par patch de la CIA et de la CIV. Chylothorax post-opératoire Switch Artériel et manoeuvre de Lecompte. Pas de sténose Aortique ou Insuffisance Aortique. Legere accélération sur les branches pulmonaires. Apparition au courant du suivi d'une obstruction sous néo-aortique de forme diaphragmatique et également d'un conus sous aortique.  Suivi dans la clinique neuro-cardio avec physiothérapie et ergothérapie  Echographie le 5 février 2015: Gradient de pic VG/Ao varie de 60 à 90mmHg , moyen varie de 35 à 43mmHg. Insuffisance aortique discrète. Qualitativement, on note de l` HVG concentrique  légère +. Dilatation de la racine aortique ( diam. 18.2mm cote Z à  2.98 ). IRM 17/02/2015: Status post-correction d'une TGV avec Switch artériel et man\x01Suvre de Lecompte. Les artères pulmonaires droite et gauche sont de bon calibre et en continuité. Toutefois, il apparaît y avoir une fine membrane juste sous la valve aortique, qui apparaît entraîner principalement l'obstruction de la voie de chasse gauche à l'échographie.  "

In [82]:
notes[256][2] = "1) Née à terme (37+4)     AVS sans complications  2) TOF rose dx anténatal avec belles branches pulmonaires     sat > 98% en néo, congé à J5 après introduction propranolol     bilan malfo N (ETF N, ophtalmo N, écho abdo petite ectasie bassinet bilatéral 5mm à G et 3mm à D)     jamais de crises hypoxiques     asymptomatique d'un point de vue cardiorespiratoire     belle croissance  3) Hernies inguinales bilatérales s/p cure chirurgicale le 15/12/2018        surveillance SIP post-op, pas de complications"

In [83]:
notes[294][2] = "induction a 38+3SA APGAR 899 PN 2.88kg CPAP en période néonatale Rashkind fait  PG en période néonatale cessé le 29/05 puis repris le 02/06 TDM thoracique 29/05/18: Situs solitus avec boucle ventriculaire droite et transposition droite des gros vaisseaux.  Discordance ventriculo-artérielle et concordance auriculo-ventriculaire.  De plus, malalignement de l'aorte avec CIV secondairement à une déviation du septum conal qui entraîne une obstruction sous-pulmonaire. Post-procédure de Rashkind. Pneumomédiastin au moins modéré.  ETF 29/05/18 : normale"

In [84]:
notes[296][2] = "2 ans 1 mois   T21    Canal AV intermédiaire CIA typeII Arc Aortique G artère sous-clavière abérante HTAP légère Retard pondérale  AP: Mère toxicomane, trouble psychiatrique, famille d'accueil adéquate ex 32 sem  PN 1770g   ETF N Bronchiolite Apnées centrales PSG 03/2014 s/p adénoïdo-amydalectomie  Rx pré-op: lanoxin, Lasix  ETT 06/2014: CIA 5.1 mm shunt G->D, CIV 6mm partiellement colmatée, gradient VG-VD 40 mmHg, courbure septale II diastole, insuffisance valvulaire minime, dilatation cavité droite   KT 2014/03 pour doute HTAP (VD-VG 80 --> 30): HTAP légère  (Qp-Qs??)  Scan cardiaque: Canal AV de type intermédiaire déjà connu avec une artère pulmonaire  principale effectivement augmentée en taille mais sans que je puisse  identifier d'anomalie parenchymateuse pulmonaire outre deux petites zones  de perfusion en mosaïque au lobe supérieur gauche et une zone  d'atélectasie du lobe supérieur droit pour expliquer l'hypertension artérielle  pulmonaire. Arc aortique gauche avec sous-clavière droite aberrante "

In [85]:
notes[1410][2] = "SOP(24/02) par Dr Vobecky Anesthésie: induction au masque avec sevo. SaO2 pré-op 82% AA. IET facile avec tube microcuff 3.0 ballonet, fixé à 11 cm à la narine D. Acide trenexamique bolus 100 mg/kg puis perfusion 10 mg/kg/h Protamine reçue Produits sanguins recus en SOP:            -> Sous pompe: 2 culots et 1/2 plasma            -> Post-Pompe: 1 plaq, 1 cryo, 70 cc culot globulaire, 60 cc plasma  Procédure: Fermeture CIV large avec patch, fermeture CIV musc avec point, switch artériel, transfert des coronaires. section et ligature CA, fermeture CIA CEC 267 min, clampage 2h. 4 fils de pace en place. Drains thoraciques en place. Plèvre D ouverte, plèvre gauche fermée. N'a pas pu avoir de MUF efficace (probleme technique) à la sortie bradycardie sinusale 80-120 traité avec pacing auriculaire à 150.  Sortie sur adre 0,03, milri bolus (50) et perf 0,5, dopa 3, NO 10, Nitro à 2 Écho cardiaque post-op (en SOP): HVG modérée avec diminution de la contractilité légère. Pas d`IM. Pas de gradient VG/Ao. IAo discrète. IT légère avec gradient VD/OD de 30 mmHg. Pas de gradient VD/AP. Large patche de CIV visualisée sans shunt résiduel. Petite CIV musculaire résiduelle avec gradient VG/VD de 30 mmHg. Flux coronarien mal visualisé. Écho de contraste négatif.  À l'arrivée SIP sur milri 0.5, epi 0.03, Nitro 0,5.  Perfusion culot 50 cc/h et plasma 50 cc/h Intubée, ventilée, sat 100%, BEA bilat. NO 10 ppm. Pace AAI 150, TA 50/30 a l'arrivée avec TOG et TVC basses: remplissage culot et PFC acceléré et bolus Ca Absence d'acidose, bonne oxygenation, lactate 1.2, SVO2 45  Sternum ouvert Saignement ad 60ml/hre par les drains thoraciques. A l'arrivé melange de culot et PFC a 100 ml/hre, acide tranexamique a 10mg/kg/hre, avons repeté protamine et tranfusé cryo et plaquettes. Pas hypothermique. iCa 1,2"

In [86]:
notes[5254][2] = "Transfert de Rouyn. IVRS en début de semaine, hospit X 3 jours, congé à domicile le 15/01 (sat 93-95% AA à ce moment, RSV/influenza nég). Traitement récent pour suspicion de pneumonie d'aspiration, Clavulin terminé en début de semaine. Le 19/01, détérioration respiratoire rapide, épisode de cyanose à la maison, pic fébrile, rapidement emmenée à l'urgence. À l'arrivée, hypotone, grisâtre, sat. 80% AA, tachycarde, tirage généralisé, hépatomégalie. Assistance avec PEEP rapidement débutée, gaz initial 7.28/54. Ceftri X 1. Tentative de Ventolin sans effet.  Écho coeur par Dr. Tak-Tak à Rouyn : HTAP isosystémique (PAP 70 mmHg), cavités droites très dilatées, IT lég-mod, VCI dilatée, dextrocardie, shunt bidi via CIA, cavités gauches normales, fonction systolique préservée mais VG comprimé par VD, impression de flux rétrograde dans veines pulmonaires sup. ?  Loading milrinone par la suite.  Cas géré entre pédiatre Rouyn & USIP CHUSJ depuis environ 19h. En bref, intubation naso-trachéale avec TET 4 ballonnet, induction Kéta + Fenta + Rocu, par anesthésiste. Support H-D avec milri 0.5 + épi 0.03 (pas de souci H-D à l'intubation). Sat. initialement autour de 80-85%, puis augmentée ad 95% sous FiO2 100%. Pas de NO de disponible. Lasix 2/kg. Gaz améliorés ad 7.37/51/28. Lactate 4.5 -> 1.5. FSC avec GB 41 / HB 102 / PLT 315. Dépistage RSV/influenza nég. ALT 290. Na+ 131 -> 135.  Transport Med-Évaq via Val-d'Or. Globalement stable pendant le vol, plusieurs bolus de fenta + rocu X 1. Sat minimale 83, sinon 95-97%, FC 160-170, TAS ad 100. 2 VVP. Dernier gaz per-transport environ 7.35/41.  Éval rapide à l'urgence puis transfert USIP."

In [87]:
notes[6588][2] = "X2jours prodrome viral avec toux, sans rhinorrhée ni congestion toutefois Pas d'apnée ni cyanose notées ni fièvre objectivée à la maison  X 24h PatientName plus amorphe, diminution de plus de 50% des boires (allaitement maternel exclusif) mère devait réveiller le PatientName pour allaitement diurèse toutefois préservée 4-5 mictions ce jour pas de vomissements, pas de diarrhées, pas de sang dans les selles  Parents consultent dans la soirée à urgence Saint-Jérôme Là-bas T 38,2R Plusieurs épisodes d'apnées 5-10sec nécessitant stimulation, 2 épisodes de brady ad 77 accompagnés de désat ad 80%  Astrup initial en capillaire 7,20/68/26 , contrôle 19h 7,30/50  Bilan septique complet fait avec PL neg et PCR HSV en cours RXP là-bas interprété N, dépistage rapide RSV influenza Neg, urine ? (résultats pas retrouvés au dossier) Début cefo et genta là-bas à doses méningées, acyclovir débutée à l'urgence ici  À l'urgence HSJ  FR 50 Sat 98% AA TAS 70 FC 150-170 3-4 épisodes d'apnées brèves 5-10sec sans désat associée Brady ad 77 X1 avec désat 85% LNHD 7LPM débutée en bas Bolus 20cc/kg X1 re: marbrée, tachycarde 170-180"

In [88]:
notes[7692][2] = 'Post-op immédiat amygdalectomie + adenoidectomie + myringotomie bilatérale sous AG Atropine/ propofol/ decadron reçus pour intubation 200cc LR + dexmedetomidine 15mg x 15 min Procédure bien tolérée Réveil spontané en salle de réveil intubé. Sat 100% FC = 90. Extubé en salle de réveil. Vers 10:30, dose morphine 3 mg IV x 1 donné au lieu de 0.3 mg alors que patient était extubé. Patient endormi initialement, mais saturation demeurait normale. Vers 12:15, debut de ronronnement et diminution de la sat à 94%. Boyeau O2 à 28% installé (Patient refusait LN et masque).  Saturation demeurait autour de 90-94%. O2 dans boyau augmenté à 60%.  A mon passage vers 14h00, patient bien éveillé et BEG. Saturation à 96% avec 60% de boyau. Pas de ronflement. Fc= 100 Fr= 20. Pas de tirage. BEA x 2. Bruits de transmission des voies aériennes hautes. Pas de sibilances. pas de crépitants  Observation à la salle de réveil x 2h. Persistance besoin O2. Désaturation ad 80% (objectivé par MD) lorsque patient dort dans les bras de la mère. Augmentation de la saturation avec repositionnement. '

In [89]:
notes[126][2] = "Grossesse sans particularité  GBS- Naissance à 40 1/7 sem- Dystocie des épaules sans complication, Apgar 8-9-9  PN 3,6kg Polypnéique avec plainte expiratoire aux boires depuis la naissance mais bonne prise de poids   Cardiomyopathie dilatée d'origine indéterminée   Hospitalisé USIP du 30/05/2013 au 25/06/2013     ECMO 30/05 au 04/06 avec vent dans VG  IRC Suivi Dr Phan  Écho rénale avril 2014    Rein D 5.5cm G 5.3 cm (peu de croissance depuis la dernière écho)  Évaluation neuro Retard développement suivi Dr Lorti Semblait s'améliorer selon note pédiatre Rouyn (Dr Cardinal) du 14/03/2014 Scan tête le 23-10-2013   Proéminence diffuse des espaces ss-arachnoidiens non spécifique. Normal par ailleurs.  Spasme du sanglot"

In [90]:
notes[4518][2] = 'Anesthésie: Induction: TET 4.5 grade I, oral Voies:JID 4.8cm, periph x2, radiale droite Saignement: minime  Chirurgie: Fontan non fenestré extracardiaque Nouvelle électrodes ventriculaire + changement boitier Plèvres ouvertes bilatérales  CEC: 91 minutes Sortie: facile, bradycardie 60 sinuale résolue avec dopamine x10minutes'

In [91]:
notes[9676][2] = "H 6 ans et demi Transféré de Mont Laurier pour bronchospasme sévère  Arrivé à l'urgence à 1h30 du matin, difficulté respiratoire, dyspnée x 23h la veille  Vu en salle de stab là bas : SaO2 75-80% à l'air libre, obnubilé  Aurait eu IVRS/toux x 4 jours, fièvre (pas mesurée), un peu céphalée, Vox1 hier (liquide), pas diarrhée, pas dlr abdo, pas de myalgies.  Avait quand même énergie (pas au lit affaissé) mais n'est pas aller à l'école   37.7 rectal. Poids estimé 22kg  IET avec l'anesthésiste TET 6.0 À reçu Solumedrol 44mg à 2h00, NS à flush, qtité? Ventolin + Atrovent en nébul, Sulfate de Mg 1.1g (50mg/kg) Ceftriaxone 1.1g (50mg/kg), bolus de D5%(0.45% ou 0.9%) NS 250cc,  Pour intubation 44mg kétamine, versed 2mg, Rocu 30mg, annectine 35mg  Puis mis sous perfusion de kétamine 1.5mg/kg/h + Ventolin IV 0.5-->1.5mcg/kg/min RX poumons : début infiltrat pms G haut du lobe inf vs bas du lobe sup, pas de latéral dispo.  Gaz là bas : 7.21-->7.12 CO2 58-->74 À l'arrivée à l'urgence : pH 7.07/CO2 89"

In [92]:
notes[6524][2] = "difficulté au boires depuis 24-48h. Plus tachypneic et détresse au boires. Sudation x 1 Pas de température, pas de toux, pas d'étouffements.  Contact + infectieux chez fraterie.  S'est présenté a Hopital St-Jerome avec sat 85% amélioré ad 100% avec 1L O2, Fc 145, afebrile, RR50 Bolus x 1 NS Cardiomegalie majeure au RXP et transfert HSJ"

In [93]:
notes[2414][2] = "Suspicion de DiGeorges (Fisch 22q11 négatif par ailleurs - patient refuse d'autres tests génétiques) Déficit immunitaire / thymectomie : 1 dose de IVIG reçu le 07/03 Obesite, Retard des acquisitions mais scolarisation normale Lésions cutanées surtout au niveau des plis (perlèche) avec culture candidose + Présence de kyste pilonidale"

In [94]:
yes= [2, 4, 22, 16, 52, 31, 23, 43, 67, 64, 32]
no = [30, 46, 47, 60, 70, 94, 157, 127, 1, 54, 9]

## 3.3 Fixing little typos related to dates format

In [95]:
#before processing
print(notes[1722])
print(notes[62][2][570:650])

['860', '3050720', '2016.07.21 : CIV PM, dilatation cavites gauches, stenoses sous pulmonaire, eperon sous aortique, FOP, CIA fermee  Rx lasix 5mg TID prevacid', 'yes']
  -Hospit 14.08 au 05.09.2014: Admis dans un contexte de SYNCOPE  avec incontine


In [96]:
for i in range(2, len(notes), 2):
    pattern = r"\d{2}\.\d{2}\.\d{2}"#full day
    result = re.finditer(pattern, notes[i][2])
    if result:
        note = list(notes[i][2])
        for m in result:
            a, b = m.start(0), m.end(0)
            note[a+2]='/'
            note[a+5]='/'
        notes[i][2] = ''.join(note)
    
    pattern = r"\d{2}\.\d{1}\.\d{2}"#full day
    result = re.finditer(pattern, notes[i][2])
    if result:
        note = list(notes[i][2])
        for m in result:
            a, b = m.start(0), m.end(0)
            note[a+2]='/'
            note[a+4]='/'
        notes[i][2] = ''.join(note)
    
    pattern = r"\d{2}\.\d{2}"#day and month
    result = re.finditer(pattern, notes[i][2])
    if result:
        note = list(notes[i][2])
        for m in result:
            a, b = m.start(0), m.end(0)
            note[a+2]='/'
        notes[i][2] = ''.join(note)

In [97]:
#after processing
print(notes[1722])
print(notes[62][2][570:650])

['860', '3050720', '2016/07/21 : CIV PM, dilatation cavites gauches, stenoses sous pulmonaire, eperon sous aortique, FOP, CIA fermee  Rx lasix 5mg TID prevacid', 'yes']
  -Hospit 14/08 au 05/09/2014: Admis dans un contexte de SYNCOPE  avec incontine


In [98]:
#removing lists items numbers
for i in range(2, len(notes), 2):
    matches = re.findall(r"(\s\d{1,2}\.\s?[A-Za-ce-z]|^\d{1,2}\.\s[A-Za-z])", notes[i][2])
    if len(matches) > 1 and re.search(r"1\.", matches[0]):
        is_enumeration, cursor = True, 1
        while is_enumeration and cursor < len(matches):
            is_enumeration = (float(matches[cursor][1:3]) == cursor+1)
            cursor += 1
        if is_enumeration:
            notes[i][2] = re.sub(r"(^|\s)\d{1,2}\.\s?([A-Za-ce-z])", r"\1item) \2", notes[i][2])
            

## 3.4 Listing the doublons

In [99]:
notes_without_doublons = [] #collectionne les id des notes sans prendre en compte les doublons
list_of_doublons = {} #collectionne les classes d'équivalence de notes
deja_vu_notes = [] #collectionne les notes de manière unique
for i in range(2, len(notes), 2):
    note = notes[i][2]
    try:
        idx = deja_vu_notes.index(note)
        list_of_doublons[idx].append(i)
    except ValueError:
        notes_without_doublons.append(notes[i])
        list_of_doublons[len(deja_vu_notes)] = [i] #l'id des classes d'équivalence est pris comme le rang d'apparition de la classe d'équivalence dans la détection des classes
        deja_vu_notes.append(note)

This line lists all duplicate notes in the dataset. For each duplicated note, the ID of its first occurrence is provided, followed by a list of note IDs corresponding to all other duplicates of that note.

(`note_id`, `[note_id of doublon_1, note_id of doublon_2, ...]`)

In [100]:
 sorted(list_of_doublons.items(), key= lambda x:len(x[1]), reverse=True)[:5]

[(103,
  [208,
   830,
   1036,
   1968,
   2256,
   4748,
   4784,
   4832,
   4902,
   5070,
   5730,
   5778,
   6218,
   6436]),
 (954, [1970, 2480, 4786, 5452, 5692, 9876]),
 (793, [1592, 1972, 2502, 10068]),
 (829, [1664, 1814, 2554, 5258]),
 (1995, [4668, 5446, 5670, 9102])]

In [101]:
len(labels), len(notes_without_doublons)

(918, 4796)

## 3.5 Looking among the non-annotated notes to see which ones have a lot of numerical data

In [102]:
votes = {k:[] for k in range(159, len(notes_without_doublons))}

In [103]:
#FE , gradient, diamètre and find intersection, Tension Artérielle, Fraction
for i in range(159, len(notes_without_doublons)):
    try:
        ids = notes_without_doublons[i][2].index('FC ')
        votes[i].append('FE')
    except ValueError:
        pass

In [104]:
for i in range(159, len(notes_without_doublons)):
    try:
        ids = notes_without_doublons[i][2].index('APP ')
        votes[i].append('APP')
    except ValueError:
        pass

In [105]:
#
for i in range(159, len(notes_without_doublons)):
    try:
        ids = notes_without_doublons[i][2].index('diamètre ')
        votes[i].append('diamètre')
    except ValueError:
        pass

In [106]:
#
for i in range(159, len(notes_without_doublons)):
    try:
        ids = notes_without_doublons[i][2].index('Fraction ')
        votes[i].append('Fraction')
    except ValueError:
        pass

In [107]:
for i in range(159, len(notes_without_doublons)):
    try:
        ids = notes_without_doublons[i][2].index('sat ')
        votes[i].append('sat')
    except ValueError:
        pass

In [108]:
votes_counts = {k: len(votes[k]) for k in range(159, len(notes_without_doublons))}

In [109]:
votes_rank = sorted(votes_counts.items(), key= lambda x:x[1], reverse=True)

In [110]:
list_of_considered_notes = [i for i in range(100)] + [107, 108, 109, 120, 122, 123, 125,127, 130, 131,132, 133, 134, 137, 140, 141, 144, 146, 147, 149, 150, 156, 158]

In [111]:
votes_rank[40:60]

[(2772, 2),
 (2773, 2),
 (2801, 2),
 (2807, 2),
 (2836, 2),
 (2870, 2),
 (2883, 2),
 (2918, 2),
 (2942, 2),
 (2977, 2),
 (3022, 2),
 (3115, 2),
 (3124, 2),
 (3128, 2),
 (3150, 2),
 (3155, 2),
 (3235, 2),
 (3241, 2),
 (3263, 2),
 (3272, 2)]

### 3.5.1. Auto-annotating some of the selected notes, to increase a little bit the dataset size

In [113]:
labels2 = []

In [114]:
labels2.append(['60', '2.6mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['60', '2mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['60', '2.1', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['69', '8,7', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['39', '22', 'gradient ventriculaire', 'mmHg'])
labels2.append(['66', '3.3', 'Taille CIA/CIV', 'mm'])
labels2.append(['76', '2.2-2.4mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['76', '5.59mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['107', '20%', 'Saturation en oxygène'])
labels2.append(['107', '80%', 'Saturation en oxygène'])
labels2.append(['108', '56mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['108', '14.6', 'Taille CIA/CIV', 'mm'])
labels2.append(['109', '8', 'Taille CIA/CIV', 'mm'])
labels2.append(['109', '34', 'gradient ventriculaire', 'mmHg'])
labels2.append(['120', '3-2-3', 'apgar'])
labels2.append(['122', '1-4-5', 'apgar'])
labels2.append(['123', "98%-100%", 'Saturation en oxygène'])
labels2.append(['123', '80-100/min', 'Fréquence cardiaque', 'bpm'])
#labels2.append(['123', '90-100/50-60', 'Tension artérielle', 'mmHg'])
labels2.append(['125', '8-9-9', 'apgar'])
labels2.append(['127', '98%', 'Saturation en oxygène'])
labels2.append(['130', '9-9-9', 'apgar'])
labels2.append(['130', '46mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['130', '4-5mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['130', '8', 'Taille CIA/CIV', 'mm'])
labels2.append(['131', '8-9-9', 'apgar'])
labels2.append(['132', '7-9-10', 'apgar'])
labels2.append(['133', '5-8-8', 'apgar'])
labels2.append(['133', '60%', 'Saturation en oxygène'])
labels2.append(['133', '60', 'Saturation en oxygène'])
labels2.append(['133', '87%', 'Saturation en oxygène'])
labels2.append(['134', '90-95%', 'Saturation en oxygène'])
labels2.append(['137', '8-8-8', 'apgar'])
labels2.append(['140', '6-9-9', 'apgar'])
labels2.append(['141', '7mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['144', '8.7', 'Taille CIA/CIV', 'mm'])
labels2.append(['144', '8.8', 'Taille CIA/CIV', 'mm'])
labels2.append(['146', '899', 'apgar'])
labels2.append(['147', '40', 'gradient ventriculaire', 'mmHg'])
labels2.append(['147', '5.1', 'Taille CIA/CIV', 'mm'])
labels2.append(['147', '6mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['147', '80', 'gradient ventriculaire', 'mmHg'])
labels2.append(['147', '30', 'gradient ventriculaire', 'mmHg'])
labels2.append(['149', '75-80%', 'Saturation en oxygène'])
labels2.append(['149', '75-85%', 'Saturation en oxygène'])
labels2.append(['149', '30', 'gradient ventriculaire', 'mmHg'])
labels2.append(['149', '30', 'gradient ventriculaire', 'mmHg'])
labels2.append(['149', '73mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['149', '51', 'gradient ventriculaire', 'mmHg'])
labels2.append(['149', '7,4', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['149', '8,5', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['149', '8,1', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['149', '5.25', 'Taille CIA/CIV', 'mm'])
labels2.append(['149', '17', 'gradient ventriculaire', 'mmHg'])
labels2.append(['149', '9.4', 'Taille CIA/CIV', 'mm'])
labels2.append(['150', '11', 'Taille CIA/CIV', 'mm'])
labels2.append(['150', '16', 'Taille CIA/CIV', 'mm'])
labels2.append(['156', '8-8-8', 'apgar'])
labels2.append(['158', '9-9-9', 'apgar'])
labels2.append(['2153', '106', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2153', '100', 'Saturation en oxygène'])
labels2.append(['2153', '30-35', "Contractibilité"])
labels2.append(['2153', '88', 'Saturation en oxygène'])
labels2.append(['2153', '129', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2153', '121', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2153', '91%', 'Saturation en oxygène'])
labels2.append(['2153', '96', 'Saturation en oxygène'])
labels2.append(['2153', '50', "Contractibilité"])
labels2.append(['2595', '4.56mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['2595', '4mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['2595', '3.5mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['425', '85-95%', 'Saturation en oxygène'])
labels2.append(['425', '55%', 'Saturation en oxygène'])
labels2.append(['425', '90-95', 'Saturation en oxygène'])
labels2.append(['425', '76', 'gradient ventriculaire', 'mmHg'])
labels2.append(['425', '3.5', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['425', '3.8', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['582', '30%', 'Saturation en oxygène'])
labels2.append(['582', '75-80%', 'Saturation en oxygène'])
labels2.append(['582', '4.1mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['582', '4-5mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['685', '2.7mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '3.2', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '2.6', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '7.6', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '2', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '3.8', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '3.3', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '3.6', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '3', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '3,7', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '2', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '7mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '5.5', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '6mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['685', '74mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['685', '11mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['704', '82%', 'Saturation en oxygène'])
labels2.append(['704', '80-120', 'Fréquence cardiaque', 'bpm'])
labels2.append(['704', '150', 'Fréquence cardiaque', 'bpm'])
#labels2.append(['704', '30', 'gradient ventriculaire', 'mmHg'])
labels2.append(['704', '100%', 'Saturation en oxygène'])
labels2.append(['704', '150', 'Fréquence cardiaque', 'bpm'])
labels2.append(['721', '75%', 'Saturation en oxygène'])
labels2.append(['721', '10mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['721', '5mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['981', '160-170', 'Fréquence cardiaque', 'bpm'])
labels2.append(['981', '85', 'Fréquence cardiaque', 'bpm'])
labels2.append(['981', '90%', 'Saturation en oxygène'])
labels2.append(['981', '6%', 'Saturation en oxygène'])
labels2.append(['981', '50', 'Fréquence cardiaque', 'bpm'])
labels2.append(['981', '80%', 'Saturation en oxygène'])
labels2.append(['1201', '30%', 'Saturation en oxygène'])
labels2.append(['1201', '75-80%', 'Saturation en oxygène'])
labels2.append(['1201', '4.1mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['1201', '4-5mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['1384', '95-105', 'Fréquence cardiaque', 'bpm'])
labels2.append(['1384', '75%', 'Saturation en oxygène'])
labels2.append(['1384', '86%', 'Saturation en oxygène'])
labels2.append(['1470', '111', 'Fréquence cardiaque', 'bpm'])
labels2.append(['1470', '30%', 'Saturation en oxygène'])
labels2.append(['1578', '180', 'Fréquence cardiaque', 'bpm'])
labels2.append(['1578', '80%', 'Saturation en oxygène'])
labels2.append(['1578', '10mmHG', 'gradient ventriculaire', 'mmHg'])
labels2.append(['1578', '9mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['1578', '67%', 'Saturation en oxygène'])
labels2.append(['1578', '27%', 'Saturation en oxygène'])
labels2.append(['1725', '170', 'Fréquence cardiaque', 'bpm'])
labels2.append(['1725', '70-75%', 'Saturation en oxygène'])
labels2.append(['1725', '3.3mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['1725', '7.8', 'Taille CIA/CIV', 'mm'])
labels2.append(['1730', '90-95', 'Fréquence cardiaque', 'bpm'])
labels2.append(['1730', '120', 'Fréquence cardiaque', 'bpm'])
labels2.append(['1730', '100%', 'Saturation en oxygène'])
labels2.append(['1730', '75%', 'Saturation en oxygène'])
labels2.append(['1730', '120', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2207', '100-110', 'gradient ventriculaire', 'mmHg'])
labels2.append(['2207', '90%', 'Saturation en oxygène'])
labels2.append(['2207', '68', 'Saturation en oxygène'])
#labels2.append(['2207', '7mm', 'Diamètre Artère Pulmonaire', 'mm'])
#labels2.append(['2207', '45mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['2207', '17mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['2208', '88-92', 'Saturation en oxygène'])
labels2.append(['2208', '20', 'gradient ventriculaire', 'mmHg'])
labels2.append(['2208', '12-14', 'gradient ventriculaire', 'mmHg'])
labels2.append(['1930', '130', 'Fréquence cardiaque', 'bpm'])
labels2.append(['1930', '4.75mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['1930', '4.4mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['1930', '5mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['1930', '3mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['2269', '8mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['2269', '25', 'gradient ventriculaire', 'mmHg'])
labels2.append(['2269', '130', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2273', '11mmHG', 'gradient ventriculaire', 'mmHg'])
labels2.append(['2273', '190-200', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2273', '55', 'Saturation en oxygène'])
labels2.append(['2273', '180-185', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2276', '160-170', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2276', '85', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2276', '90%', 'Saturation en oxygène'])
labels2.append(['2276', '6%', 'Saturation en oxygène'])
labels2.append(['2276', '50', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2276', '80%', 'Saturation en oxygène'])
labels2.append(['2475', '3-8-9', 'apgar'])
labels2.append(['2475', '97', 'Saturation en oxygène'])
labels2.append(['2475', '70-88', 'Saturation en oxygène'])
labels2.append(['2475', '160', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2475', '80-83', 'Saturation en oxygène'])
labels2.append(['2485', '5.6mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['2485', '65%', 'Saturation en oxygène'])
labels2.append(['2485', '89%', 'Saturation en oxygène'])
labels2.append(['2485', '9.25mm', 'Taille CIA/CIV', 'mm'])
labels2.append(['2485', '35%-40%', 'Contractibilité'])
labels2.append(['2485', '6.9mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['2485', '5mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['2485', '154', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2514', '200', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2514', '80%', 'Saturation en oxygène'])
labels2.append(['2514', '90-96%', 'Saturation en oxygène'])
labels2.append(['2514', '25%', "Contractibilité"])
labels2.append(['2514', '130-140', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2610', '65-72%', 'Saturation en oxygène'])
labels2.append(['2610', '170', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2616', '80-90%', 'Saturation en oxygène'])
labels2.append(['2616', '3', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['2616', '4', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['2616', '7', 'Taille CIA/CIV', 'mm'])
labels2.append(['2616', '12', 'gradient ventriculaire', 'mmHg'])
labels2.append(['2626', '93-95%', 'Saturation en oxygène'])
labels2.append(['2626', '80%', 'Saturation en oxygène'])
labels2.append(['2626', '80-85%', 'Saturation en oxygène'])
labels2.append(['2626', '95%', 'Saturation en oxygène'])
labels2.append(['2626', '95-97%', 'Saturation en oxygène'])
labels2.append(['2626', '160-170', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2700', '45', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2700', '40%', 'Saturation en oxygène'])
labels2.append(['2700', '50-55', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2700', '50%', 'Saturation en oxygène'])
labels2.append(['2700', '45', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2700', '80', 'Fréquence cardiaque', 'bpm'])

In [115]:
labels2.append(['2917', '195', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2917', '100%', 'Saturation en oxygène'])
labels2.append(['2917', '80', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2917', '83%', 'Saturation en oxygène'])
labels2.append(['2959', '44%', 'Saturation en oxygène'])
labels2.append(['2959', '80', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2959', '180', 'Fréquence cardiaque', 'bpm'])
labels2.append(['2959', '97%', 'Saturation en oxygène'])
labels2.append(['3240', '93%', 'Saturation en oxygène'])
labels2.append(['3240', '115', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3240', '80%', 'Saturation en oxygène'])
labels2.append(['3240', '94%', 'Saturation en oxygène'])
labels2.append(['3240', '100-110', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3240', '95%', 'Saturation en oxygène'])
labels2.append(['3245', '171', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3245', '96%', 'Saturation en oxygène'])
labels2.append(['3245', '130', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3245', '100%', 'Saturation en oxygène'])
labels2.append(['3292', '50', 'Saturation en oxygène'])
labels2.append(['3292', '50-60', 'Saturation en oxygène'])
labels2.append(['3292', '50', 'Saturation en oxygène'])
labels2.append(['3292', '60', 'Saturation en oxygène'])
labels2.append(['3292', '70%', 'Saturation en oxygène'])
labels2.append(['3292', '40', 'Saturation en oxygène'])
labels2.append(['3293', '80%', 'Saturation en oxygène'])
labels2.append(['3293', '77', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3293', '98%', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3293', '150-170', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3293', '77', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3293', '85%', 'Saturation en oxygène'])
labels2.append(['3293', '170-180', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3326', '85%', 'Saturation en oxygène'])
labels2.append(['3326', '160', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3326', '150', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3326', '96%', 'Saturation en oxygène'])
labels2.append(['3332', '11mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['3332', '13mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['3332', '3.7mm', 'Diamètre Artère Pulmonaire', 'mm'])
labels2.append(['3332', '21mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['3369', '92', 'Saturation en oxygène'])
labels2.append(['3369', '140', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3369', '94', 'Saturation en oxygène'])
labels2.append(['3369', '92', 'Saturation en oxygène'])
#labels2.append(['3425', '4mmHg', 'gradient ventriculaire', 'mmHg'])
#labels2.append(['3425', '4mmHg', 'gradient ventriculaire', 'mmHg'])
#labels2.append(['3425', '2mmHg', 'gradient ventriculaire', 'mmHg'])
labels2.append(['3425', '80', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3425', '160/min', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3425', '14%', 'Contractibilité'])
labels2.append(['3425', '40%', "Contractibilité"])
labels2.append(['3425', '99%', 'Saturation en oxygène'])
labels2.append(['3425', '14%', 'Contractibilité'])
labels2.append(['3463', '98-99%', 'Saturation en oxygène'])
labels2.append(['3463', '55%', 'Saturation en oxygène'])
labels2.append(['3463', '97%', 'Saturation en oxygène'])
labels2.append(['3463', '172', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3463', '97%', 'Saturation en oxygène'])
labels2.append(['3463', '173', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3582', '87', 'Saturation en oxygène'])
labels2.append(['3582', '91%', 'Saturation en oxygène'])
labels2.append(['3582', '124', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3582', '95-96', 'Saturation en oxygène'])
labels2.append(['3739', '60', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3739', '78%', 'Saturation en oxygène'])
labels2.append(['3739', '100%', 'Saturation en oxygène'])
labels2.append(['3739', '114', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3845', '100%', 'Saturation en oxygène'])
labels2.append(['3845', '90', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3845', '94%', 'Saturation en oxygène'])
labels2.append(['3845', '90-94%', 'Saturation en oxygène'])
labels2.append(['3845', '96%', 'Saturation en oxygène'])
labels2.append(['3845', '100', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3845', '80%', 'Saturation en oxygène'])
labels2.append(['3869', '80%', 'Saturation en oxygène'])
labels2.append(['3869', '200', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3869', '91-94%', 'Saturation en oxygène'])
labels2.append(['3869', '85%', 'Saturation en oxygène'])
labels2.append(['3878', '155', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3878', '145', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3878', '160', 'Fréquence cardiaque', 'bpm'])
labels2.append(['3878', '50-55%', "Contractibilité"])

In [116]:
list_of_considered_notes += [2153, 2595, 425, 582, 685, 704, 721, 981, 1201, 1384, 1470, 1578, 1725, 1730, 2207, 2208, 1930, 2269, 2273, 2276, 2475, 2485, 2514, 2610, 2616, 2626, 2700]

In [117]:
list_of_considered_notes += [2917, 2959, 3240, 3245, 3292, 3293, 3326, 3332, 3369, 3425, 3463, 3582, 3739, 3845, 3869, 3878]

## 3.6. Fixing some minor tokens issues in the annotations provided by the Clinicians

In [112]:
#fixing some splitted tokens
labels[439][1] = '65%'
labels[478][1] = '46%'
labels[815][1] = '27%'
labels[284][1] = '31.3%'
labels[310][1] = '40,7%'
labels[480][1] = '29%'
labels[100][1] = '4,5mm'
labels[175][1] = '5.5mm'
labels[176][1] = '6.6mm'
labels[381][1] = '2.8mm'
labels[382][1] = '2.3mm'
labels[486][1] = '4.9mm'
labels[487][1] = '5.7mm'
labels[655][1] = '18mm'
labels[4][1] = '80-85%'
labels[19][1] = '87%'
labels[165][1] = '85-88%'
labels[166][1] = '75%'
labels[224][1] = '50-65%'
labels[228][1] = '70-75%'
labels[394][1] = '25%'
labels[401][1] = '96%'
labels[529][1] = '80-85%'
labels[543][1] = '85-90%'
labels[550][1] = '65%'
labels[601][1] = '65-85%'
labels[603][1] = '75%'
labels[738][1] = '92%'
labels[13][1] = '8-9-9'
labels[24][1] = '8-9-9'
labels[328][1] = '9-9-10'
labels[340][1] = '8-9'
labels[396][1] = '8-9'
labels[417][1] = '1-2-3'
labels[448][1] = '9-9-9-'
labels[545][1] = '7-9-10'
labels[563][1] = '8-9-9'
labels[572][1] = '7-8-10'
labels[583][1] = '7-8-8'
labels[644][1] = '8-9-9'
labels[674][1] = '9-9-9'
labels[724][1] = '8-9-9'
labels[739][1] = '8/8/9'
labels[752][1] = '5.5.7'
labels[764][1] = '6/7/9'
labels[777][1] = '9/9/9'
labels[845][1] = '6/8/8'
labels[913][1] = '9-9-9'
labels[6][1] = '6,5'
labels[7][1] = '7,1'
labels[538][1] = '1,6'
labels[886][1] = '60-68'
labels[296][1] = '2.4mm'
labels[329][1] = '2.5mm'
labels[489][1] = '6.5mm'
labels[773][1] = '4mm'
labels[185][1] = '4.4mm'
labels[186][1] = '3.5mm'
labels[297][1] = '2mm'
labels[390][1] = '10mm'
labels[490][1] = '7mm'
labels[586][1] = '4.4mm'
labels[587][1] = '2.7'
labels[774][1] = '7mm'
labels[792][1] = '10mm'
labels[3][1] = '50-60mmHg'
labels[11][1] = '87mmHg'
labels[178][1] = '27mmHg'
labels[179][1] = '28mmHg'
labels[180][1] = '41mmHg'
labels[206][1] = '28mmHg'
labels[287][1] = '75mmHg'
labels[528][1] = '89mmHG'
labels[368][1] = '6,8'
labels[465][1] = '7,7'
labels[159][1] = '25%'

# 4. Structuring the dataset

## 4.1. Breaking down the notes in tokens and regrouping + indexing the categories

In [108]:
texts = {}
for i in list_of_considered_notes:
    texts[i] = notes[2*i+2][2].split()

In [109]:
for i in list_of_considered_notes:
    clean_text = []
    for word in texts[i]:
        cleaned_words = remove_punctuation(word)
        clean_text.append(cleaned_words)
    texts[i] = sum(clean_text, [])

In [110]:
attr_to_class = {"Fraction d'éjection":"Contractibilité",
                "Valeur de la fraction d'éjection en Simson":"Contractibilité",
                "Fraction de raccourcissement": "Contractibilité",
                'Fréquence cardiaque : bradycardie':'Fréquence cardiaque',
                'Diamètre Artère Pulmonaire Droite distale':'Diamètre Artère Pulmonaire',
                'Diamètre Artère Pulmonaire Droite proximale': 'Diamètre Artère Pulmonaire',
                'Diamètre Artère Pulmonaire Gauche proximale': 'Diamètre Artère Pulmonaire',
                'Diamètre Artère Pulmonaire Principale': 'Diamètre Artère Pulmonaire',
                'Diamètre Artère Pulmonaire Droite': 'Diamètre Artère Pulmonaire',
                'Diamètre Artère Pulmonaire Gauche': 'Diamètre Artère Pulmonaire',
                'diamètre Artère Pulmonaire': 'Diamètre Artère Pulmonaire',
                'diamètre Artère Pulmonaire Droite': 'Diamètre Artère Pulmonaire',
                'Saturation pulsée en oxygène': 'Saturation en oxygène',
                'Valeur Saturation Pulsée en Oxygène': 'Saturation en oxygène',
                'saturation veineuse en oxygène': 'Saturation en oxygène',
                'Valeur de la Saturation Pulsée en oxygène': 'Saturation en oxygène',
                'saturation artérielle en oxygène': 'Saturation en oxygène',
                'Objectif cible de Saturation en Oxygène': 'Saturation en oxygène',
                'Valeur Fraction inspirée en Oxygène': 'Saturation en oxygène',
                'score apgar à 1 minute': 'apgar',
                'score apgar à 10 minutes': 'apgar',
                'score apgar à 5 minutes': 'apgar',
                "score d'apgar (à une minute et cinq minutes)": 'apgar',
                "score d'apgar (à une minute, cinq minutes et 10 minutes)": 'apgar',
                'gradient ventricule droit-ventricule gauche artère pulmonaire': 'gradient ventriculaire',
                'Gradient ventricule droit-artère pulmonaire': 'gradient ventriculaire',
                "gradient de pression entre l'artère pulmonaire droite et le ventricule droit":'gradient ventriculaire',
                "gradient de pression entre l'artère pulmonaire gauche et le ventricule droit": 'gradient ventriculaire',
                "gradient de pression entre le ventricule gauche et l'artère pulmonaire": 'gradient ventriculaire',
                "gradient de pression entre l'artère pulmonaire principale et le ventricule droit": 'gradient ventriculaire',
                "Gradient Ventricule Gauche - Ventricule Droit": 'gradient ventriculaire',
                "Gradient de pic Ventricule Gauche - Ventricule Droit": 'gradient ventriculaire',
                "Gradient valve pulmonaire": 'gradient ventriculaire',
                "Gradiant max Valve pulmonaire": 'gradient ventriculaire',
                'Taille Communication Inter Auriculaire': 'Taille CIA/CIV',
                'Taille de la Communication Inter Ventriculaire': 'Taille CIA/CIV',
                'diamètre de la communication interauriculaire': 'Taille CIA/CIV' 
                } 

In [111]:
class_to_index = {"Contractibilité": 1,
                  'Fréquence cardiaque': 2,
                  'Diamètre Artère Pulmonaire': 3,
                  'Saturation en oxygène': 4,
                  'apgar': 5,
                  'gradient ventriculaire': 6,
                  'Taille CIA/CIV': 7
}

In [113]:
lines_to_ignore = [5, 26, 116, 121, 131, 144, 160, 316, 341, 356, 357, 379, 397, 405, 406, 418, 419, 449, 450, 522, 530, 537, 544, 546, 547, 564, 565, 573, 574, 584, 585, 605, 606, 607, 608, 645, 646, 648, 675, 676, 704, 725, 726, 740, 741, 753, 754, 765, 766, 778, 779, 795, 801, 846, 847, 887, 914, 915]

#I think for 316 it's diameter instead of taille de la CIV

## 4.2. Annotating each tokens (labels and tokens alignment)

In [ ]:
classes = {k:[] for k in list_of_considered_notes}
for i in range(len(labels)):
    if labels[i][2] in attr_to_class:
        classe = attr_to_class[labels[i][2]]
        if not (i in lines_to_ignore):
            patient_id = int(labels[i][0])
            classes[patient_id].append((labels[i][1], classe))

In [ ]:
for i in range(len(labels2)):
    classe = labels2[i][2]
    patient_id = int(labels2[i][0])
    classes[patient_id].append((labels2[i][1], classe))

### 4.2.1. Finding some notes where some numerical values are duplicated and associating to each occurance the right class one by one

In [116]:
pos_classes = {k:np.zeros(len(texts[k]), dtype=int) for k in list_of_considered_notes}
print("list of doublons")
for i in list_of_considered_notes:
    for (value, classe) in classes[i]:
        if not value == '9 -9-10': #special case to deal with later
            index_class = class_to_index[classe]
            index = texts[i].index(value)
            indices = [k for k in range(len(texts[i])) if texts[i][k]==value]
            if len(indices)> 1:
                print(i, 'value=', value, 'class=', index_class, 'pos=', indices)
            else:
                pos_classes[i][index] = index_class
        else:
            index_class = class_to_index[classe]
            index = texts[i].index('-9-10')
            pos_classes[i][index] = index_class
            pos_classes[i][index-1] = index_class

list of doublons
2 value= 8 class= 7 pos= [56, 76]
43 value= 7 class= 5 pos= [23, 25]
43 value= 7 class= 5 pos= [23, 25]
52 value= 24 class= 1 pos= [143, 224]
52 value= 23 class= 1 pos= [135, 216]
69 value= 11 class= 3 pos= [136, 182]
69 value= 11 class= 3 pos= [136, 182]
69 value= 10 class= 3 pos= [144, 187]
69 value= 10 class= 7 pos= [144, 187]
70 value= 5.5 class= 3 pos= [100, 106]
149 value= 30 class= 6 pos= [108, 195]
149 value= 30 class= 6 pos= [108, 195]
150 value= 11 class= 7 pos= [19, 136]
685 value= 2 class= 3 pos= [30, 117, 145]
685 value= 2 class= 3 pos= [30, 117, 145]
685 value= 7mm class= 3 pos= [179, 193]
704 value= 150 class= 2 pos= [157, 291]
704 value= 150 class= 2 pos= [157, 291]
1730 value= 120 class= 2 pos= [7, 38]
1730 value= 120 class= 2 pos= [7, 38]
2700 value= 45 class= 2 pos= [52, 260]
2700 value= 45 class= 2 pos= [52, 260]
3245 value= 100% class= 4 pos= [86, 309]
3292 value= 50 class= 4 pos= [195, 280]
3292 value= 50 class= 4 pos= [195, 280]
3292 value= 40 cl

In [205]:
#dealing with numerical values doublons in single notes
pos_classes[2][56] = 7
pos_classes[43][23] = 5
pos_classes[43][25] = 5
pos_classes[52][143] = 1
pos_classes[52][224] = 1
pos_classes[52][135] = 1
pos_classes[52][216] = 1
pos_classes[69][136] = 3
pos_classes[69][182] = 3
pos_classes[69][144] = 3
pos_classes[69][187] = 7
pos_classes[70][106] = 3
pos_classes[149][108] = 6
pos_classes[149][195] = 6
pos_classes[149][22] = 5 #apgar
pos_classes[149][24] = 5 #apgar
pos_classes[149][26] = 5 #apgar
pos_classes[150][19] = 7
pos_classes[685][30] = 3
pos_classes[685][117] = 3
pos_classes[685][193] = 3
pos_classes[704][157] = 2
pos_classes[704][291] = 2
#pos_classes[704][216] = 6
pos_classes[1730][7] = 2
pos_classes[1730][38] = 2
pos_classes[2700][52] = 2
pos_classes[2700][260] = 2
pos_classes[3245][309] = 4
pos_classes[3292][195] = 4
pos_classes[3292][280] = 4
pos_classes[3292][343] = 4
pos_classes[3293][91] = 2
pos_classes[3293][174] = 2
pos_classes[3369][65] = 4
pos_classes[3369][107] = 4
pos_classes[3425][303] = 1
pos_classes[3425][421] = 1
#pos_classes[3425][40] = 6
#pos_classes[3425][116] = 6
pos_classes[3463][119] = 4
pos_classes[3463][147] = 4

In [206]:
pos_classes[52]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## 4.3. Breaking down long notes into smaller notes

In [207]:
dataset = []
minimal_sentence_size = 7
for i in list_of_considered_notes:
    text = texts[i]
    if len(text) < minimal_sentence_size:
        continue
    class_indexes = pos_classes[i]
    #listing the breaking points while avoiding breaks like "Dr. Fournier."
    breaks = [-1]
    for j in range(len(text)):
        if (text[j] == '.') and (j > breaks[-1] + minimal_sentence_size):
            breaks.append(j)
    if breaks[-1]!= len(text)-1:
        breaks.append(len(text)-1)
    
    #checks if the last sentence has the minimal length
    if breaks[-1]-breaks[-2] < minimal_sentence_size:
        breaks.pop(-2)
        
    for j in range(len(breaks)-1):
        sample = {'tokens': text[breaks[j]+1: breaks[j+1]+1], 
                  'classes': class_indexes[breaks[j]+1: breaks[j+1]+1],
                  'extracted_from': i}
        dataset.append(sample)

In [208]:
len(dataset)

1052

In [209]:
dataset[5]

{'tokens': ['Écho',
  'cardiaque',
  '(',
  '14/08',
  ')',
  ':',
  'gradient',
  'VD-VG',
  'AP',
  'de',
  '50-60mmHg',
  '.'],
 'classes': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0]),
 'extracted_from': 1}

## 4.4. Splitting the dataset into training, testing, validating while ensuring equal distribution of classes accross datasets

In [210]:
class_to_sample = {1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[]}
for i in range(len(dataset)):
    sample = dataset[i]
    for class_idx in sample['classes']:
        if class_idx != 0:
            class_to_sample[class_idx].append(i)

In [211]:
class_to_sample= {k:np.array(v) for k,v in class_to_sample.items()}
{k:len(v) for (k,v) in class_to_sample.items()}

{1: 18, 2: 65, 3: 57, 4: 114, 5: 52, 6: 38, 7: 53}

In [212]:
def compute_loss(mask, display= False):
    loss = 0
    for i in range(1, 8):
        counters = np.zeros(3)
        n = 0
        for j in mask[class_to_sample[i]]:
            if j > -1:
                counters[j] += 1
                n += 1
        probs = counters/n
        if display:
            print(probs)
        kl = .15*np.log(probs[0]/.15) + .15*np.log(probs[1]/.15) + .7*np.log(probs[2]/.7)
        kl = kl if n > 0 else -np.inf
        loss -= kl
    return loss

In [215]:
mask_arr = categorical(torch.tensor([.15, .15, .7]), len(dataset))
#mask_arr = -np.ones(len(dataset),dtype=int)

In [216]:
for i in range(len(dataset)):
    choice_corresponding_losses = np.empty(3)# gives the corresponding losses for each choice of classification of the sample i
    mask_arr[i] = 0
    choice_corresponding_losses[0] = compute_loss(mask_arr)
    
    mask_arr[i] = 1
    choice_corresponding_losses[1] = compute_loss(mask_arr)
    
    mask_arr[i] = 2
    choice_corresponding_losses[2] = compute_loss(mask_arr)
    
    choice = np.argmin(choice_corresponding_losses)
    mask_arr[i] = choice

Now we deal with samples which does not contain annoted numerical values

In [217]:
empty_samples = []
for i in range(len(dataset)):
    if sum(dataset[i]['classes']) == 0:
        empty_samples.append(i)
empty_samples = np.array(empty_samples)
len(empty_samples)

802

In [218]:
for i in range(len(empty_samples)):
    if (i%20==6) or (i%20==12) or (i%20==18):
        mask_arr[empty_samples[i]] = 0
    
    elif (i%20==3) or (i%20==9) or (i%20==15):
        mask_arr[empty_samples[i]] = 1
    
    else:
        mask_arr[empty_samples[i]] = 2

In [219]:
#with open('mask_array.npy', 'wb') as f:
#    np.save(f, mask_arr)
    
with open('mask_array.npy', 'rb') as f:
    mask_arr = np.load(f)

In [220]:
compute_loss(mask_arr, True)

[0.16666667 0.16666667 0.66666667]
[0.15384615 0.15384615 0.69230769]
[0.15789474 0.14035088 0.70175439]
[0.14912281 0.14912281 0.70175439]
[0.15384615 0.15384615 0.69230769]
[0.15789474 0.15789474 0.68421053]
[0.1509434  0.1509434  0.69811321]


0.003949408748836987

In [221]:
train_ds = []
val_ds = []
test_ds = []
for i in range(len(dataset)):
    if mask_arr[i]==0:
        test_ds.append(dataset[i])
    elif mask_arr[i]==1:
        val_ds.append(dataset[i])
    else:
        train_ds.append(dataset[i])

In [222]:
print(len(test_ds), len(val_ds), len(train_ds))

159 156 737


### 4.4.1. Saving everything

In [223]:
with open("sadcsip/test", "wb") as fp:   #Pickling
   pickle.dump(test_ds, fp)
 
#with open("test", "rb") as fp:   # Unpickling
#   test_ds = pickle.load(fp)

with open("sadcsip/val", "wb") as fp:   #Pickling
   pickle.dump(val_ds, fp)
 
#with open("val", "rb") as fp:   # Unpickling
#   val_ds = pickle.load(fp)

with open("sadcsip/train", "wb") as fp:   #Pickling
   pickle.dump(train_ds, fp)
 
#with open("train", "rb") as fp:   # Unpickling
#   train_ds = pickle.load(fp)

## 4.5. Minor post processing

In [229]:
with open("sadcsip/test", "rb") as fp:   # Unpickling
    test_ds = pickle.load(fp)
 
with open("sadcsip/val", "rb") as fp:   # Unpickling
    val_ds = pickle.load(fp)
 
with open("sadcsip/train", "rb") as fp:   # Unpickling
    train_ds = pickle.load(fp)
    
edited_test_ds = []
edited_val_ds = []
edited_train_ds = []

for sample in test_ds:
    edited_text = []
    new_classes = []
    for i in range(len(sample['tokens'])):
        word = sample['tokens'][i]
        edited_word = minor_edit(word).split()
        if len(edited_word) > 1:
            edited_classes = []
            for token in edited_word:
                """
                if re.search(r"[%\-\+A-Za-z/:x\|><X\*~«]", token[0]):
                    edited_classes.append(0)
                else:
                    edited_classes.append(sample['classes'][i])
                """
                if re.search(r"^(\d+[\.,]\d+|\d+)$", token):
                    edited_classes.append(sample['classes'][i])
                else:
                    edited_classes.append(0)

            #if sample['classes'][i] != 0:
            #    print( word, edited_word, edited_classes, "extracted:", sample['extracted_from'])
        else:
            edited_classes = [sample['classes'][i]]
        edited_text.append(edited_word)
        new_classes.extend(edited_classes)

    edited_sample = {'tokens': sum(edited_text, []), 'classes': new_classes, 'extracted_from': sample['extracted_from']}
    edited_test_ds.append(edited_sample)

for sample in val_ds:
    edited_text = []
    new_classes = []
    for i in range(len(sample['tokens'])):
        word = sample['tokens'][i]
        edited_word = minor_edit(word).split()
        if len(edited_word) > 1:
            edited_classes = []
            for token in edited_word:
                if re.search(r"^(\d+[\.,]\d+|\d+)$", token):
                    edited_classes.append(sample['classes'][i])
                else:
                    edited_classes.append(0)

            #if sample['classes'][i] != 0:
            #    print( word, edited_word, edited_classes, "extracted:", sample['extracted_from'])
        else:
            edited_classes = [sample['classes'][i]]
        edited_text.append(edited_word)
        new_classes.extend(edited_classes)

    edited_sample = {'tokens': sum(edited_text, []), 'classes': new_classes, 'extracted_from': sample['extracted_from']}
    edited_val_ds.append(edited_sample)
    
for sample in train_ds:
    edited_text = []
    new_classes = []
    for i in range(len(sample['tokens'])):
        word = sample['tokens'][i]
        edited_word = minor_edit(word).split()
        if len(edited_word) > 1:
            edited_classes = []
            for token in edited_word:
                if re.search(r"^(\d+[\.,]\d+|\d+)$", token):
                    edited_classes.append(sample['classes'][i])
                else:
                    edited_classes.append(0)
            #if sample['classes'][i] != 0:
            #    print( word, edited_word, edited_classes, "extracted:", sample['extracted_from'])
        else:
            edited_classes = [sample['classes'][i]]
        edited_text.append(edited_word)
        new_classes.extend(edited_classes)

    edited_sample = {'tokens': sum(edited_text, []), 'classes': new_classes, 'extracted_from': sample['extracted_from']}
    edited_train_ds.append(edited_sample)

In [230]:
with open("sadcsip/test", "wb") as fp:   #Pickling
   pickle.dump(edited_test_ds, fp)
 
with open("sadcsip/val", "wb") as fp:   #Pickling
   pickle.dump(edited_val_ds, fp)
 
with open("sadcsip/train", "wb") as fp:   #Pickling
   pickle.dump(edited_train_ds, fp)

In [231]:
edited_train_ds[4]['tokens']

['Écho',
 'cardiaque',
 '(',
 '14',
 '/',
 '08',
 ')',
 ':',
 'gradient',
 'VD-VG',
 'AP',
 'de',
 '50',
 '-',
 '60',
 'mmHg',
 '.']

# 5. Build Datasets with customly tokenized numbers

In [232]:
with open("sadcsip/test", "rb") as fp:   # Unpickling
    test_ds = pickle.load(fp)
 
with open("sadcsip/val", "rb") as fp:   # Unpickling
    val_ds = pickle.load(fp)
 
with open("sadcsip/train", "rb") as fp:   # Unpickling
    train_ds = pickle.load(fp)
    
tokenized_test_ds = []
tokenized_val_ds = []
tokenized_train_ds = []

for sample in test_ds:
    tokenized_text = []
    for word in sample['tokens']:
        tokenized_word = custom_tokenizer(word)
        tokenized_text.append(tokenized_word)
        
    tokenized_sample = {'tokens': tokenized_text, 'classes': sample['classes'], 'extracted_from': sample['extracted_from']}
    tokenized_test_ds.append(tokenized_sample)

for sample in val_ds:
    tokenized_text = []
    for word in sample['tokens']:
        tokenized_word = custom_tokenizer(word)
        tokenized_text.append(tokenized_word)
        
    tokenized_sample = {'tokens': tokenized_text, 'classes': sample['classes'], 'extracted_from': sample['extracted_from']}
    tokenized_val_ds.append(tokenized_sample)

for sample in train_ds:
    tokenized_text = []
    for word in sample['tokens']:
        tokenized_word = custom_tokenizer(word)
        tokenized_text.append(tokenized_word)
        
    tokenized_sample = {'tokens': tokenized_text, 'classes': sample['classes'], 'extracted_from': sample['extracted_from']}
    tokenized_train_ds.append(tokenized_sample)

In [233]:
tokenized_train_ds[4]['tokens']

['Écho',
 'cardiaque',
 '(',
 '1D4U',
 '/',
 '0D8U',
 ')',
 ':',
 'gradient',
 'VD-VG',
 'AP',
 'de',
 '5D0U',
 '-',
 '6D0U',
 'mmHg',
 '.']

In [234]:
with open("sadcsip/tokenized_test", "wb") as fp:   #Pickling
   pickle.dump(tokenized_test_ds, fp)
 
with open("sadcsip/tokenized_val", "wb") as fp:   #Pickling
   pickle.dump(tokenized_val_ds, fp)
 
with open("sadcsip/tokenized_train", "wb") as fp:   #Pickling
   pickle.dump(tokenized_train_ds, fp)

# 6. Build Datasets for QA

In [21]:
with open("sadcsip/test", "rb") as fp:   # Unpickling
    test_ds = pickle.load(fp)
 
with open("sadcsip/val", "rb") as fp:   # Unpickling
    val_ds = pickle.load(fp)
 
with open("sadcsip/train", "rb") as fp:   # Unpickling
    train_ds = pickle.load(fp)
    
qa_test_ds = []
qa_val_ds = []
qa_train_ds = []

for sample in test_ds:
    for i in range(len(sample['tokens'])):
        if re.search(r"^(\d+[,\.]\d+|\d+\.?)$", sample['tokens'][i]):
            qa_text = ["text:"] + sample['tokens']
            qa_text = qa_text + "\n A quelle catégorie appartient".split() + [sample['tokens'][i]]
            qa_text = qa_text + "parmi 1 = Contractibilité, 2 = Fréquence cardiaque, 3 = Diamètre Artère Pulmonaire, \
                  4 = Saturation en oxygène, 5 = apgar, 6 = gradient ventriculaire, 7 = Taille CIA/CIV, 0 = aucune".split()
            
            qa_sample = {'text': qa_text, 'label': sample['classes'][i], 'extracted_from': sample['extracted_from']}
            qa_test_ds.append(qa_sample)

for sample in val_ds:
    for i in range(len(sample['tokens'])):
        if re.search(r"^(\d+[,\.]\d+|\d+\.?)$", sample['tokens'][i]):
            qa_text = ["text:"] + sample['tokens']
            qa_text = qa_text + "\n A quelle catégorie appartient".split() + [sample['tokens'][i]]
            qa_text = qa_text + "parmi 1 = Contractibilité, 2 = Fréquence cardiaque, 3 = Diamètre Artère Pulmonaire, \
                  4 = Saturation en oxygène, 5 = apgar, 6 = gradient ventriculaire, 7 = Taille CIA/CIV, 0 = aucune".split()
    
            qa_sample = {'text': qa_text, 'label': sample['classes'][i], 'extracted_from': sample['extracted_from']}
            qa_val_ds.append(qa_sample)

for sample in train_ds:
    for i in range(len(sample['tokens'])):
        if re.search(r"^(\d+[,\.]\d+|\d+\.?)$", sample['tokens'][i]):
            qa_text = ["text:"] + sample['tokens']
            qa_text = qa_text + "\n A quelle catégorie appartient".split() + [sample['tokens'][i]]
            qa_text = qa_text + "parmi 1 = Contractibilité, 2 = Fréquence cardiaque, 3 = Diamètre Artère Pulmonaire, \
                  4 = Saturation en oxygène, 5 = apgar, 6 = gradient ventriculaire, 7 = Taille CIA/CIV, 0 = aucune".split()
        
            qa_sample = {'text': qa_text, 'label': sample['classes'][i], 'extracted_from': sample['extracted_from']}
            qa_train_ds.append(qa_sample)


In [22]:
with open("sadcsip/qa_test", "wb") as fp:   #Pickling
   pickle.dump(qa_test_ds, fp)
 
with open("sadcsip/qa_val", "wb") as fp:   #Pickling
   pickle.dump(qa_val_ds, fp)
 
with open("sadcsip/qa_train", "wb") as fp:   #Pickling
   pickle.dump(qa_train_ds, fp)

# 7. Build Blind Dataset (for first paper) 

In [122]:
with open("sadcsip/test", "rb") as fp:   # Unpickling
    test_ds = pickle.load(fp)
 
with open("sadcsip/val", "rb") as fp:   # Unpickling
    val_ds = pickle.load(fp)
 
with open("sadcsip/train", "rb") as fp:   # Unpickling
    train_ds = pickle.load(fp)

blind_test_ds = []
blind_val_ds = []
blind_train_ds = []

for sample in test_ds:
    blind_text = []
    for token in sample['tokens']:
        #if sample['classes'][i] != 0:
        #    blind_sample['tokens'][i] = 'nombre'
        blind_text.append(number_masking(token))
    blind_sample = {'tokens': blind_text, 'classes': sample['classes'].copy(), 'extracted_from': sample['extracted_from']}
    blind_test_ds.append(blind_sample)
            
for sample in val_ds:
    blind_text = []
    for token in sample['tokens']:
        #if sample['classes'][i] != 0:
        #    blind_sample['tokens'][i] = 'nombre'
        blind_text.append(number_masking(token))
    blind_sample = {'tokens': blind_text, 'classes': sample['classes'].copy(), 'extracted_from': sample['extracted_from']}
    blind_val_ds.append(blind_sample)
    
for sample in train_ds:
    blind_text = []
    for token in sample['tokens']:
        #if sample['classes'][i] != 0:
        #    blind_sample['tokens'][i] = 'nombre'
        blind_text.append(number_masking(token))
    blind_sample = {'tokens': blind_text, 'classes': sample['classes'].copy(), 'extracted_from': sample['extracted_from']}
    blind_train_ds.append(blind_sample)


In [123]:
blind_train_ds[4]['tokens']

['Écho',
 'cardiaque',
 '(',
 'nombre',
 '/',
 'nombre',
 ')',
 ':',
 'gradient',
 'VD-VG',
 'AP',
 'de',
 'nombre',
 '-',
 'nombre',
 'mmHg',
 '.']

In [124]:
with open("sadcsip/blind_test", "wb") as fp:   #Pickling
   pickle.dump(blind_test_ds, fp)
 
#with open("sadcsip/blind_test", "rb") as fp:   # Unpickling
#   blind_test_ds = pickle.load(fp)

with open("sadcsip/blind_val", "wb") as fp:   #Pickling
   pickle.dump(blind_val_ds, fp)
 
#with open("sadcsip/blind_val", "rb") as fp:   # Unpickling
#   blind_val_ds = pickle.load(fp)

with open("sadcsip/blind_train", "wb") as fp:   #Pickling
   pickle.dump(blind_train_ds, fp)
 
#with open("sadcsip/blind_train", "rb") as fp:   # Unpickling
#   blind_train_ds = pickle.load(fp)

In [125]:
print(len(blind_train_ds), len(blind_val_ds), len(blind_test_ds))

737 156 159


# 8. Build Blind Dataset (for second paper) Xval dataset

In [33]:
min_exponents = -4

In [42]:
with open("sadcsip/test", "rb") as fp:   # Unpickling
    test_ds = pickle.load(fp)
 
with open("sadcsip/val", "rb") as fp:   # Unpickling
    val_ds = pickle.load(fp)
 
with open("sadcsip/train", "rb") as fp:   # Unpickling
    train_ds = pickle.load(fp)

xval_test_ds = []
xval_val_ds = []
xval_train_ds = []

for sample in test_ds:
    blind_text = []
    h_nums = []
    significands = []
    exponents = []
    for token in sample['tokens']:
        t, h = number_blinding(token)
        significand, exponent = scientific_notiation(h)
        blind_text.append(t)
        h_nums.append(h)
        significands.append(significand)
        exponents.append(exponent - min_exponents)
        #if (blind_text[-1] != token) and (blind_text[-1] != 'nombre'):
        #    print(token, blind_text[-1], sample['extracted_from'])
        #if blind_text[-1] == 'nombre':
        #    print(token)

    blind_sample = {'tokens': blind_text, 'classes': sample['classes'].copy(),
        'h_nums': h_nums, 'significands': significands, 'exponents': exponents, 
        'extracted_from': sample['extracted_from']}
    xval_test_ds.append(blind_sample)
            
for sample in val_ds:
    blind_text = []
    h_nums = []
    significands = []
    exponents = []
    for token in sample['tokens']:
        t, h = number_blinding(token)
        significand, exponent = scientific_notiation(h)
        blind_text.append(t)
        h_nums.append(h)
        significands.append(significand)
        exponents.append(exponent - min_exponents)
        #if (blind_text[-1] != token) and (blind_text[-1] != 'nombre'):
        #    print(token, blind_text[-1], sample['extracted_from'])
        #if blind_text[-1] == 'nombre':
        #    print(token)

    blind_sample = {'tokens': blind_text, 'classes': sample['classes'].copy(),
        'h_nums': h_nums, 'significands': significands, 'exponents': exponents, 
        'extracted_from': sample['extracted_from']}
    xval_val_ds.append(blind_sample)
    
for sample in train_ds:
    blind_text = []
    h_nums = []
    significands = []
    exponents = []
    for token in sample['tokens']:
        t, h = number_blinding(token)
        significand, exponent = scientific_notiation(h)
        blind_text.append(t)
        h_nums.append(h)
        significands.append(significand)
        exponents.append(exponent - min_exponents)
        #if (blind_text[-1] != token) and (blind_text[-1] != 'nombre'):
        #    print(token, blind_text[-1], sample['extracted_from'])
        #if blind_text[-1] == 'nombre':
        #    print(token)

    blind_sample = {'tokens': blind_text, 'classes': sample['classes'].copy(),
        'h_nums': h_nums, 'significands': significands, 'exponents': exponents, 
        'extracted_from': sample['extracted_from']}
    xval_train_ds.append(blind_sample)


In [43]:
xval_train_ds[4]

{'tokens': ['Écho',
  'cardiaque',
  '(',
  'NUM',
  '/',
  'NUM',
  ')',
  ':',
  'gradient',
  'VD-VG',
  'AP',
  'de',
  'NUM',
  '-',
  'NUM',
  'mmHg',
  '.'],
 'classes': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 6, 0, 0],
 'h_nums': [1, 1, 1, 14.0, 1, 8.0, 1, 1, 1, 1, 1, 1, 50.0, 1, 60.0, 1, 1],
 'significands': [1.0,
  1.0,
  1.0,
  1.4,
  1.0,
  8.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  5.0,
  1.0,
  6.0,
  1.0,
  1.0],
 'exponents': [4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4],
 'extracted_from': 1}

In [44]:
with open("sadcsip/xval_test", "wb") as fp:   #Pickling
   pickle.dump(xval_test_ds, fp)
 
with open("sadcsip/xval_val", "wb") as fp:   #Pickling
   pickle.dump(xval_val_ds, fp)
 
with open("sadcsip/xval_train", "wb") as fp:   #Pickling
   pickle.dump(xval_train_ds, fp)

In [45]:
print(len(xval_train_ds), len(xval_val_ds), len(xval_test_ds))

737 156 159


# 9. Creation of the unlabeled dataset for MLM task

In [126]:
texts = []
for i in range(2, len(notes), 2):
    texts.append(re.sub(r"(^|\s)(\d+)(q|p)\s(\d)", r"\1\2\3\4", notes[i][2]).split())

for i in range(len(texts)):
    clean_text = []
    for word in texts[i]:
        cleaned_words = remove_punctuation(word)
        clean_text.append(cleaned_words)
    texts[i] = sum(clean_text, [])
    
dataset = []
minimal_sentence_size = 10
for i in range(len(texts)):
    text = texts[i]
    if len(text) < minimal_sentence_size:
        continue
    
    #listing the breaking points while avoiding breaks like "Dr. Fournier."
    breaks = [-1]
    for j in range(len(text)):
        if (text[j] == '.') and (j > breaks[-1] + minimal_sentence_size):
            breaks.append(j)
    if breaks[-1]!= len(text)-1:
        breaks.append(len(text)-1)
    
    #checks if the last sentence has the minimal length
    if breaks[-1]-breaks[-2] < minimal_sentence_size:
        breaks.pop(-2)
    
    for j in range(len(breaks)-1):
        sample = {'tokens': text[breaks[j]+1: breaks[j+1]+1], 
                  'extracted_from': i}
        #minor post editing
        edited_text = []
        for word in sample['tokens']:
            edited_word = minor_edit(word).split()
            #if re.search(r"«", word):
            #    edited_word = minor_edit(word)
            #    print(word, edited_word, sample['extracted_from'])
            edited_text.append(edited_word)
        
        edited_sample = {'tokens': sum(edited_text, []), 'extracted_from': sample['extracted_from']}
    
        dataset.append(edited_sample)

In [127]:
len(dataset)

22238

In [128]:
rand = np.random.rand(len(dataset))
mask_arr = (rand < 0.2)
mlm_train_ds = []
mlm_val_ds = []
for i in range(len(dataset)):
    if mask_arr[i]:
        mlm_val_ds.append(dataset[i])
    else:
        mlm_train_ds.append(dataset[i])

In [129]:
print(len(mlm_train_ds), len(mlm_val_ds))

17818 4420


In [130]:
with open("sadcsip/mlm_val", "wb") as fp:   #Pickling
   pickle.dump(mlm_val_ds, fp)
 
#with open("sadcsip/mlm_val", "rb") as fp:   # Unpickling
#   mlm_val_ds = pickle.load(fp)

with open("sadcsip/mlm_train", "wb") as fp:   #Pickling
   pickle.dump(mlm_train_ds, fp)
 
#with open("sadcsip/mlm_train", "rb") as fp:   # Unpickling
#   mlm_train_ds = pickle.load(fp)

# 10. Creation of the unlabeled dataset for MLM task Xval

In [131]:
min_exponents = -4

In [132]:
with open("sadcsip/mlm_val", "rb") as fp:   # Unpickling
   mlm_val_ds = pickle.load(fp)
 
with open("sadcsip/mlm_train", "rb") as fp:   # Unpickling
   mlm_train_ds = pickle.load(fp)

xval_mlm_val_ds = []
xval_mlm_train_ds = []
all_exponents = []

for sample in mlm_val_ds:
    blind_text = []
    h_nums = []
    significands = []
    exponents = []
    for token in sample['tokens']:
        t, h = number_blinding(token)
        significand, exponent = scientific_notiation(h)
        blind_text.append(t)
        h_nums.append(h)
        significands.append(significand)
        exponents.append(exponent - min_exponents)
        all_exponents.append(exponent)
        #if (blind_text[-1] != token) and (blind_text[-1] != 'nombre'):
        #    print(token, blind_text[-1], sample['extracted_from'])
        #if blind_text[-1] == 'nombre':
        #    print(token)

    blind_sample = {'tokens': blind_text,
        'h_nums': h_nums, 'significands': significands, 'exponents': exponents, 
        'extracted_from': sample['extracted_from']}
    xval_mlm_val_ds.append(blind_sample)
            
for sample in mlm_train_ds:
    blind_text = []
    h_nums = []
    significands = []
    exponents = []
    for token in sample['tokens']:
        t, h = number_blinding(token)
        significand, exponent = scientific_notiation(h)
        blind_text.append(t)
        h_nums.append(h)
        significands.append(significand)
        exponents.append(exponent - min_exponents)
        all_exponents.append(exponent)
        #if (blind_text[-1] != token) and (blind_text[-1] != 'nombre'):
        #    print(token, blind_text[-1], sample['extracted_from'])
        #if blind_text[-1] == 'nombre':
        #    print(token)

    blind_sample = {'tokens': blind_text,
        'h_nums': h_nums, 'significands': significands, 'exponents': exponents, 
        'extracted_from': sample['extracted_from']}
    xval_mlm_train_ds.append(blind_sample)


In [133]:
min(all_exponents)

-4

In [134]:
with open("sadcsip/xval_mlm_val", "wb") as fp:   #Pickling
   pickle.dump(xval_mlm_val_ds, fp)
 
with open("sadcsip/xval_mlm_train", "wb") as fp:   #Pickling
   pickle.dump(xval_mlm_train_ds, fp)

# 11. Creation of the unlabeled tokenized dataset for MLM task

In [136]:
 with open("sadcsip/mlm_val", "rb") as fp:   # Unpickling
    val_ds = pickle.load(fp)
 
with open("sadcsip/mlm_train", "rb") as fp:   # Unpickling
    train_ds = pickle.load(fp)
    
mlm_tokenized_val_ds = []
mlm_tokenized_train_ds = []
mlm_tokenized_global_ds = []

for sample in val_ds:
    tokenized_sample = {k:v for (k,v) in sample.items()}
    tokenized_sample['tokens'] = [custom_tokenizer(word) for word in sample['tokens']]
    mlm_tokenized_val_ds.append(tokenized_sample)
    mlm_tokenized_global_ds.append(tokenized_sample)

for sample in train_ds:
    tokenized_sample = {k:v for (k,v) in sample.items()}
    tokenized_sample['tokens'] = [custom_tokenizer(word).split() for word in sample['tokens']]
    mlm_tokenized_train_ds.append(tokenized_sample)
    mlm_tokenized_global_ds.append(tokenized_sample)

In [137]:
with open("sadcsip/mlm_tokenized_val", "wb") as fp:   #Pickling
   pickle.dump(mlm_tokenized_val_ds, fp)
 
with open("sadcsip/mlm_tokenized_train", "wb") as fp:   #Pickling
   pickle.dump(mlm_tokenized_train_ds, fp)

with open("sadcsip/mlm_tokenized_global", "wb") as fp:   #Pickling
   pickle.dump(mlm_tokenized_global_ds, fp)

In [138]:
len(mlm_tokenized_global_ds)

22238

In [139]:
print(len(mlm_tokenized_train_ds), len(mlm_tokenized_val_ds))

17818 4420


# 12. Creation of the ComNumDataset

In [24]:
import random
len_dataset = 200000
candidate_numbers = [k for k in range(len_dataset)]
dataset = []

In [25]:
for k in range(len_dataset//4):
    selected_numbers = random.sample(candidate_numbers, k=2)
    text = aux(str(selected_numbers[0]/1000)) + ' est supérieur à ' + aux(str(selected_numbers[1]/1000)) + '.'
    label = selected_numbers[0] > selected_numbers[1]
    dataset.append({'text': text, 'label': label})

    for number in selected_numbers:
        candidate_numbers.remove(number)

    selected_numbers = random.sample(candidate_numbers, k=2)
    text = aux(str(selected_numbers[0]/1000)) + ' est inférieur à ' + aux(str(selected_numbers[1]/1000)) + '.'
    label = selected_numbers[0] < selected_numbers[1]
    dataset.append({'text': text, 'label': label})
    for number in selected_numbers:
        candidate_numbers.remove(number)

In [26]:
len(dataset)

100000

In [27]:
rand = np.random.rand(len(dataset))
mask_arr = (rand < 0.2)
mlm_train_ds = []
mlm_val_ds = []
for i in range(len(dataset)):
    if mask_arr[i]:
        mlm_val_ds.append(dataset[i])
    else:
        mlm_train_ds.append(dataset[i])

In [28]:
print(len(mlm_train_ds), len(mlm_val_ds))

80096 19904


In [ ]:
with open("mlm_val_3", "wb") as fp:   #Pickling
   pickle.dump(mlm_val_ds, fp)
 
with open("mlm_train_3", "wb") as fp:   #Pickling
   pickle.dump(mlm_train_ds, fp)
 

# 13. Tokenized version of E3C dataset

Download the french corpus of E3C dataset [here](https://github.com/hltfbk/E3C-Corpus/tree/v2.0.0) prior, if needed

In [28]:
import json
import os
from tqdm import tqdm

directories = ['layer1', 'layer2', 'layer3']
dataset = []
for directory in directories:
    path = os.path.join('E3C-Corpus/data_collection/French', directory)
    for filename in tqdm(os.listdir(path)):
        f = os.path.join(path, filename)
        file = open(f)
        data = json.load(file)
        raw_text = re.sub("\.(\.|\s)*\.", '', data["text"])
        raw_text = re.sub(r"(\D{3})(\d{1,2})\s(\d{3})\s(\d{3})(\D{3})", r"\1\2\3\4\5", raw_text) #detecting 7+ digits numbers with whitespace
        raw_text = re.sub(r"(\D{3})(\d{1,3})\s(\d{3})(\D{3})", r"\1\2\3\4", raw_text) # detecting 4+ digits numbers with whitespace
        raw_text = raw_text.split()
        preprocessed_text = []
        for word in raw_text:
            cleaned_words = remove_punctuation(word)
            tokenized_cleaned_words = []
            try:
                for cleaned_word in cleaned_words:
                    tokenized_cleaned_words.append(custom_tokenizer(cleaned_word).split())
                    #if len(tokenized_cleaned_words[-1])>1:
                    #    print("'", cleaned_word, "'", tokenized_cleaned_words[-1], "extracted:", f)
                cleaned_words = sum(tokenized_cleaned_words, [])

            except IndexError:
                print(cleaned_words, f)
            #except Exception:
            #    print(cleaned_words, f)
            preprocessed_text.append(cleaned_words)
        dataset.append(sum(preprocessed_text, []))
        file.close()

100%|██████████| 25740/25740 [24:05<00:00, 17.81it/s]


In [29]:
# creating the whole dataset first
e3c_mlm_tokenized_global_ds = []
minimal_sentence_size = 390
for text in dataset:
    #listing the breaking points while avoiding breaks like "Dr. Fournier."
    breaks = [-1]
    for j in range(len(text)):
        if (text[j] == '.') and (j > breaks[-1] + minimal_sentence_size):
            breaks.append(j)
    if breaks[-1]!= len(text)-1:
        breaks.append(len(text)-1)

    #checks if the last sentence has the minimal length
    if breaks[-1]-breaks[-2] < minimal_sentence_size:
        breaks.pop(-2)
        
    for j in range(len(breaks)-1):
        e3c_mlm_tokenized_global_ds.append({'tokens': text[breaks[j]+1: breaks[j+1]+1]})

with open("e3c_mlm_tokenized_global", "wb") as fp:   #Pickling
   pickle.dump(e3c_mlm_tokenized_global_ds, fp)

In [30]:
rand = np.random.rand(len(e3c_mlm_tokenized_global_ds))
mask_arr = (rand < 0.2)
e3c_mlm_tokenized_train_ds = []
e3c_mlm_tokenized_val_ds = []
for i in range(len(e3c_mlm_tokenized_global_ds)):
    if mask_arr[i]:
        e3c_mlm_tokenized_val_ds.append(e3c_mlm_tokenized_global_ds[i])
    else:
        e3c_mlm_tokenized_train_ds.append(e3c_mlm_tokenized_global_ds[i])

In [31]:
with open("e3c_mlm_tokenized_val", "wb") as fp:   #Pickling
   pickle.dump(e3c_mlm_tokenized_val_ds, fp)
 
with open("e3c_mlm_tokenized_train", "wb") as fp:   #Pickling
   pickle.dump(e3c_mlm_tokenized_train_ds, fp)

In [32]:
print(len(e3c_mlm_tokenized_train_ds), len(e3c_mlm_tokenized_val_ds), )

132950 33041


: 